In [ ]:
#重要！下载未来数据，2015-2100，ssp126，ssp245，ssp370，ssp585

In [ ]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_chunk(url, start, end, save_dir, file_name, chunk_num, chunk_size, retries=3):
    headers = {'Range': f'bytes={start}-{end}'}
    chunk_path = os.path.join(save_dir, f"{file_name}.part{chunk_num}")

    while retries > 0:
        try:
            response = requests.get(url, headers=headers, stream=True)
            if response.status_code in [200, 206]:
                content = response.content
                if len(content) == chunk_size:
                    with open(chunk_path, 'wb') as f:
                        f.write(content)
                    return True
                else:
                    print(f"Chunk {chunk_num} size mismatch, expected {chunk_size}, got {len(content)}")
            else:
                print(f"Chunk {chunk_num} download failed with status code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading chunk {chunk_num}: {e}")

        retries -= 1
        print(f"Retrying chunk {chunk_num}, attempt {3 - retries}")

    print(f"Failed to download chunk {chunk_num} after multiple retries.")
    return False

def download_file(url, save_dir, file_name, num_chunks=4, num_threads=20):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, file_name)

    response = requests.head(url)
    if response.status_code != 200:
        print(f"Error: Unable to access URL, status code {response.status_code}")
        return

    file_size = int(response.headers.get('content-length', 0))
    chunk_size = file_size // num_chunks
    last_chunk_size = file_size - chunk_size * (num_chunks - 1)

    futures = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for i in range(num_chunks):
            start = chunk_size * i
            end = start + chunk_size - 1 if i != num_chunks - 1 else file_size - 1
            size = chunk_size if i != num_chunks - 1 else last_chunk_size
            future = executor.submit(download_chunk, url, start, end, save_dir, file_name, i, size)
            futures.append(future)

    # 确保所有块都已正确下载
    if not all(future.result() for future in as_completed(futures)):
        print(f"Not all chunks of {file_name} were downloaded successfully.")
        return

    # 合并文件块
    try:
        with open(save_path, 'wb') as final_file:
            for i in range(num_chunks):
                part_file_path = os.path.join(save_dir, f"{file_name}.part{i}")
                if os.path.exists(part_file_path):
                    with open(part_file_path, 'rb') as part_file:
                        final_file.write(part_file.read())
                    os.remove(part_file_path)
                else:
                    print(f"Missing chunk {i} for file {file_name}")
                    return
    except Exception as e:
        print(f"Error while merging file {file_name}: {e}")

def download_yearly_data(data_type, start_year, end_year, model_name, scenario, num_threads=20):
    for year in range(start_year, end_year + 1):
        file_name = f"{data_type}_day_{model_name}_{scenario}_r1i1p1f1_gn_{year}.nc"
        base_url = f"https://ds.nccs.nasa.gov/thredds/fileServer/AMES/NEX/GDDP-CMIP6/{model_name}/{scenario}/r1i1p1f1/{data_type}/{file_name}"
        save_dir = os.path.join("M:/GDDP-CMIP6", model_name, scenario, data_type)
        print(f"Downloading {data_type} data for year: {year}")
        download_file(base_url, save_dir, file_name, num_threads=num_threads)


# 下载 tas 数据
download_yearly_data("tas", 2092, 2100, "CMCC-CM2-SR5", "ssp370", num_threads=20)

# 下载 tas 数据
download_yearly_data("tas", 2015, 2100, "CMCC-CM2-SR5", "ssp585", num_threads=20)

In [ ]:
#重要！下载历史数据 1990-2014

In [ ]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_chunk(url, start, end, save_dir, file_name, chunk_num, chunk_size, retries=3):
    headers = {'Range': f'bytes={start}-{end}'}
    chunk_path = os.path.join(save_dir, f"{file_name}.part{chunk_num}")

    while retries > 0:
        try:
            response = requests.get(url, headers=headers, stream=True)
            if response.status_code in [200, 206]:
                content = response.content
                if len(content) == chunk_size:
                    with open(chunk_path, 'wb') as f:
                        f.write(content)
                    return True
                else:
                    print(f"Chunk {chunk_num} size mismatch, expected {chunk_size}, got {len(content)}")
            else:
                print(f"Chunk {chunk_num} download failed with status code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading chunk {chunk_num}: {e}")

        retries -= 1
        print(f"Retrying chunk {chunk_num}, attempt {3 - retries}")

    print(f"Failed to download chunk {chunk_num} after multiple retries.")
    return False

def download_file(url, save_dir, file_name, num_chunks=4, num_threads=20):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, file_name)

    response = requests.head(url)
    if response.status_code != 200:
        print(f"Error: Unable to access URL, status code {response.status_code}")
        return

    file_size = int(response.headers.get('content-length', 0))
    chunk_size = file_size // num_chunks
    last_chunk_size = file_size - chunk_size * (num_chunks - 1)

    futures = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for i in range(num_chunks):
            start = chunk_size * i
            end = start + chunk_size - 1 if i != num_chunks - 1 else file_size - 1
            size = chunk_size if i != num_chunks - 1 else last_chunk_size
            future = executor.submit(download_chunk, url, start, end, save_dir, file_name, i, size)
            futures.append(future)

    # 确保所有块都已正确下载
    if not all(future.result() for future in as_completed(futures)):
        print(f"Not all chunks of {file_name} were downloaded successfully.")
        return

    # 合并文件块
    try:
        with open(save_path, 'wb') as final_file:
            for i in range(num_chunks):
                part_file_path = os.path.join(save_dir, f"{file_name}.part{i}")
                if os.path.exists(part_file_path):
                    with open(part_file_path, 'rb') as part_file:
                        final_file.write(part_file.read())
                    os.remove(part_file_path)
                else:
                    print(f"Missing chunk {i} for file {file_name}")
                    return
    except Exception as e:
        print(f"Error while merging file {file_name}: {e}")

def download_historical_data(data_type, start_year, end_year, model_name, num_threads=20):
    for year in range(start_year, end_year + 1):
        file_name = f"{data_type}_day_{model_name}_historical_r1i1p1f1_gn_{year}.nc"
        base_url = f"https://ds.nccs.nasa.gov/thredds/fileServer/AMES/NEX/GDDP-CMIP6/{model_name}/historical/r1i1p1f1/{data_type}/{file_name}"
        save_dir = os.path.join("M:/GDDP-CMIP6", model_name, "historical", data_type)
        print(f"Downloading {data_type} historical data for year: {year}")
        download_file(base_url, save_dir, file_name, num_threads=num_threads)

def download_future_data(data_type, start_year, end_year, model_name, num_threads=20):
    for year in range(start_year, end_year + 1):
        file_name = f"{data_type}_day_{model_name}_r1i1p1f1_gn_{year}.nc"
        base_url = f"https://ds.nccs.nasa.gov/thredds/fileServer/AMES/NEX/GDDP-CMIP6/{model_name}/r1i1p1f1/{data_type}/{file_name}"
        save_dir = os.path.join("M:/GDDP-CMIP6", model_name, data_type)
        print(f"Downloading {data_type} future data for year: {year}")
        download_file(base_url, save_dir, file_name, num_threads=num_threads)

# 下载 tas 数据（历史）
download_historical_data("tas", 1990, 2014, "ACCESS-CM2", num_threads=20)

# 下载 tas 数据
download_yearly_data("tas", 2015, 2100, "CMCC-CM2-SR5", "ssp585", num_threads=20)

In [ ]:
#重要！检查有问题的数据(历史)

In [ ]:
import os
import xarray as xr
from tqdm import tqdm

# 输入文件夹路径、输出文件夹路径、模式列表和情景
input_folder = r"L:\GDDP-CMIP6"
scenarios =["historical"]
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 定义一个函数来检查文件是否可以被xarray打开
def check_file(file_path):
    try:
        with xr.open_dataset(file_path, decode_times=False, engine='netcdf4'):
            return True
    except Exception as e:
        return False

# 遍历每个情景、模型和年份，检查文件
problematic_files = []
for scenario in scenarios:
    for model in tqdm(models):
        for year in range(1950, 2015):
            model_path = os.path.join(input_folder, model, scenario, "tas")

            # 根据模型构建文件名
            if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
            elif model in ["FGOALS-g3"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
            elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
            elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
            elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
            else:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"

            file_path = os.path.join(model_path, file_name_pattern)

            if not os.path.exists(file_path):
                print(f"File not found: {file_path}")
            elif not check_file(file_path):
                problematic_files.append(file_path)

# 输出有问题的文件列表
print("Problematic files:")
for file in problematic_files:
    print(file)

In [ ]:
#重要！检查有问题的数据(未来，4情景)

In [ ]:
import os
import xarray as xr
from tqdm import tqdm

# 输入文件夹路径、输出文件夹路径、模式列表和情景
input_folder = r"L:\GDDP-CMIP6"
scenarios = ["ssp126", "ssp245", "ssp370", "ssp585"]
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 定义一个函数来检查文件是否可以被xarray打开
def check_file(file_path):
    try:
        with xr.open_dataset(file_path, decode_times=False, engine='netcdf4'):
            return True
    except Exception as e:
        return False

# 遍历每个情景、模型和年份，检查文件
problematic_files = []
for scenario in scenarios:
    for model in tqdm(models):
        for year in range(2015, 2101):
            model_path = os.path.join(input_folder, model, scenario, "tas")

            # 根据模型构建文件名
            if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
            elif model in ["FGOALS-g3"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
            elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
            elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
            elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
            else:
                file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"

            file_path = os.path.join(model_path, file_name_pattern)

            if not os.path.exists(file_path):
                print(f"File not found: {file_path}")
            elif not check_file(file_path):
                problematic_files.append(file_path)

# 输出有问题的文件列表
print("Problematic files:")
for file in problematic_files:
    print(file)

In [ ]:
#这段代码是可以用的，计算了历史月度均值。

In [ ]:
import xarray as xr
import numpy as np
import os

# 定义情景和年份
scenarios = ['historical']
years = list(range(1990, 2015))

# 处理每个情景和年份
for scenario in scenarios:
    for year in years:
        # 构建文件路径
        input_dir = f'L:/CMIP6-1final/{scenario}/'
        input_file = f'tas_day_{scenario}_r1i1p1f1_gn_{year}.nc'
        file_path = os.path.join(input_dir, input_file)

        # 创建输出文件夹
        output_dir = f'L:/CMIP6-1yue/{scenario}/'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # 打开数据集
        data = xr.open_dataset(file_path)

        # 计算每个月的平均值
        monthly_mean = data.resample(time='1M').mean(dim='time', skipna=True)

        # 修改时间维度的格式
        monthly_mean['time'] = np.arange(12)

        # 保存月平均数据到NetCDF文件
        output_file = f'tas_day_{scenario}_r1i1p1f1_gn_{year}_month.nc'
        output_file_path = os.path.join(output_dir, output_file)
        monthly_mean.to_netcdf(output_file_path)

        print(f'Processed {scenario} for year {year}')

In [ ]:
#这段代码是可以用的，计算了未来月度均值。

In [ ]:
import xarray as xr
import numpy as np
import os

# 定义情景和年份
scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
years = list(range(2015, 2030)) + list(range(2030, 2101, 10))

# 遍历情景和年份
for scenario in scenarios:
    for year in years:
        # 构建文件路径
        input_file = f'L:/CMIP6-1final/{scenario}/tas_day_{scenario}_r1i1p1f1_gn_{year}.nc'
        output_dir = f'L:/CMIP6-1yue/{scenario}/'
        output_file = f'{output_dir}tas_day_{scenario}_r1i1p1f1_gn_{year}_month.nc'

        # 创建目标文件夹
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # 打开数据集
        data = xr.open_dataset(input_file)

        # 计算每个月的平均值
        monthly_mean = data.resample(time='1M').mean(dim='time', skipna=True)

        # 将温度转换为摄氏度
        monthly_mean -= 273.15

        # 修改时间维度的格式
        monthly_mean['time'] = np.arange(12)

        # 保存月平均数据到NetCDF文件
        monthly_mean.to_netcdf(output_file)

        # 关闭数据集
        data.close()


In [ ]:
#重要！这个不是计算过程！在计算26个模式每日均值时，还会遇到问题returned non-zero exit status 1，用下面代码可以检查出有问题的数据文件。

In [ ]:
import os
import subprocess
from tqdm import tqdm
import uuid

def is_cdo_available():
    try:
        subprocess.run(["wsl", "cdo", "-V"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

if not is_cdo_available():
    print("CDO is not installed or not found in PATH in WSL. Please install CDO in WSL and ensure it is in the PATH.")
    exit()

# 输入文件夹的Windows路径
input_folder = "L:/GDDP-CMIP6"

# 模型列表
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 情景和年份
#scenario = "historical"
#year = 2010

# 情景列表
scenarios = ["ssp126", "ssp245", "ssp370", "ssp585"]

# 年份范围
years = range(2015, 2101)

# 构建文件路径
files_to_check = []
for model in models:
    if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
        file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
    elif model in ["FGOALS-g3"]:
        file_name = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
    elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
        file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
    elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
        file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
    elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
        file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
    else:
        file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"

    file_path = f"L:/GDDP-CMIP6/{model}/{scenario}/tas/{file_name}"
    files_to_check.append(file_path)

# 检查每个文件
for file in tqdm(files_to_check):
    temp_output_file = f"/mnt/l/{uuid.uuid4()}.nc"  # 使用UUID生成唯一的临时文件名
    wsl_file_path = file.replace("L:", "/mnt/l").replace("\\", "/")
    try:
        subprocess.run(["wsl", "cdo", "ensmean", wsl_file_path, temp_output_file], check=True, stderr=subprocess.PIPE)
        print(f"File processed successfully: {file}")
        os.remove(temp_output_file.replace("/mnt/l", "L:"))  # 删除生成的临时文件
    except subprocess.CalledProcessError as e:
        print(f"Error processing file: {file}")
        print(f"Error message: {e.stderr.decode()}")

In [ ]:
#不要的！计算9个模式每日均值

In [ ]:
import xarray as xr
import os
from tqdm import tqdm

# 设置输入文件夹路径、输出文件夹路径、模式列表和情景
input_folder = r"L:\GDDP-CMIP6"
output_folder = r"L:\CMIP6\ssp585"
scenario = "ssp585"
models = [
    "ACCESS-CM2",
    "BCC-CSM2-MR",
    "CanESM5",
    "CMCC-CM2-SR5",
    "CMCC-ESM2",
    "FGOALS-g3",
    "GISS-E2-1-G",
    "KACE-1-0-G",
    "MRI-ESM2-0"
]

# 创建输出文件夹
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 遍历每个年份
for year in tqdm(range(2015, 2101)):
    # 创建一个空的 xarray 数据集来存储每天的模式均值数据
    daily_means = []

    # 遍历每个模型
    for model in models:
        # 获取模型文件夹路径
        model_path = os.path.join(input_folder, model, scenario)

        # 获取文件名
        # 根据模型不同的文件命名方式，使用不同的匹配规则
        if model == "FGOALS-g3":
            file_name_pattern = f"tasmax_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
        elif model == "GISS-E2-1-G":
            file_name_pattern = f"tasmax_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
        elif model == "KACE-1-0-G":
            file_name_pattern = f"tasmax_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
        else:
            file_name_pattern = f"tasmax_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"

        # 构建完整文件路径
        file_path = os.path.join(model_path, file_name_pattern)

        # 确保文件存在
        if os.path.exists(file_path):
            # 读取数据并明确指定日历类型
            ds = xr.open_dataset(file_path, decode_times=False, engine='netcdf4')
            ds['time'] = xr.cftime_range(str(year), periods=len(ds['time']), freq='D')
            ds = xr.decode_cf(ds)

            # 将时间维度设为 'date'
            ds = ds.rename({'time': 'date'})

            # 将数据添加到列表中
            daily_means.append(ds['tasmax'])
        else:
            print(f"File not found for {model} in year {year}. Skipping for this model.")

    if daily_means:
        # 合并所有模型的数据
        combined_data = xr.concat(daily_means, dim="model")

        # 计算每个日期的模式均值
        daily_model_means = combined_data.mean(dim='model', skipna=True)

        # 按照日期先后顺序排序
        daily_model_means = daily_model_means.sortby('date')

        # 保存结果到文件
        output_file = os.path.join(output_folder, f"tasmax_day_ssp585_r1i1p1f1_gn_{year}.nc")
        daily_model_means.to_netcdf(output_file)

        print(f"Year {year} processed and saved.")
    else:
        print(f"No data found for year {year}. Skipping.")

In [ ]:
#重要！这个还有问题！计算26个模式每日均值，每年大概需要12分钟

In [ ]:
import xarray as xr
import os
from tqdm import tqdm

# 设置输入文件夹路径、输出文件夹路径、模式列表和情景
input_folder = r"L:\GDDP-CMIP6"
output_folder = r"L:\CMIP6-1\ssp126"
scenario = "ssp126"
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 创建输出文件夹
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 遍历每个年份
for year in tqdm(range(2081, 2101)):
    # 创建一个空的 xarray 数据集来存储每天的模式均值数据
    daily_means = []

    # 遍历每个模型
    for model in models:
        # 获取模型文件夹路径
        model_path = os.path.join(input_folder, model, scenario, "tas")

        # 构建文件名
        if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
            file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
        elif model in ["FGOALS-g3"]:
            file_name_pattern = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
        elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
            file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
        elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
            file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
        elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
            file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
        else:
            file_name_pattern = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"


       # 构建完整文件路径
        file_path = os.path.join(model_path, file_name_pattern)

        # 确保文件存在
        if os.path.exists(file_path):
            # 读取数据并明确指定日历类型
            ds = xr.open_dataset(file_path, decode_times=False, engine='netcdf4')
            ds['time'] = xr.cftime_range(str(year), periods=len(ds['time']), freq='D')
            ds = xr.decode_cf(ds)

            # 将时间维度设为 'date'
            ds = ds.rename({'time': 'date'})

            # 将数据添加到列表中
            daily_means.append(ds['tas'])
        else:
            print(f"File not found for {model} in year {year}. Skipping for this model.")

    if daily_means:
        # 合并所有模型的数据
        combined_data = xr.concat(daily_means, dim="model")

        # 计算每个日期的模式均值
        daily_model_means = combined_data.mean(dim='model', skipna=True)

        # 按照日期先后顺序排序
        daily_model_means = daily_model_means.sortby('date')

        # 保存结果到文件
        output_file = os.path.join(output_folder, f"tas_day_ssp126_r1i1p1f1_gn_{year}.nc")
        daily_model_means.to_netcdf(output_file)

        print(f"Year {year} processed and saved.")
    else:
        print(f"No data found for year {year}. Skipping.")

In [ ]:
#重要！检查原始数据是否缺失部分日期数据,12月

In [ ]:
import os
import subprocess

def check_for_december_27st(input_folder, models, scenarios, year):
    """
    检查指定模型、情景和特定年份的NetCDF文件是否包含12月27日的数据。
    """
    for model in models:
        for scenario in scenarios:
            # 构建文件路径
            model_path = os.path.join(input_folder, model, scenario, "tas")

            # 根据模型和情景构建文件名
            if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
            elif model in ["FGOALS-g3"]:
                file_name = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
            elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
            elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
            elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
            else:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"

            file_path = os.path.join(model_path, file_name)
            wsl_file_path = file_path.replace("L:", "/mnt/l").replace("\\", "/")

            if os.path.exists(file_path):
                try:
                    # 使用CDO的showdate命令显示文件中的日期
                    result = subprocess.run(["wsl", "cdo", "showdate", wsl_file_path], 
                                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, check=True)
                    dates = result.stdout.strip().split()
                    december_27st = f"{year}-12-27"
                    if december_27st in dates:
                        print(f"December 27st is present in {file_path}")
                    else:
                        print(f"December 27st is missing in {file_path}")
                except subprocess.CalledProcessError as e:
                    print(f"Error processing file {file_path}: {e}")
            else:
                print(f"File not found: {file_path}")

# 输入文件夹的Windows路径
input_folder = "L:/GDDP-CMIP6"

# 模型列表
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 情景列表
scenarios = ["ssp126", "ssp245", "ssp370", "ssp585"]

# 只检查2025年的数据
check_for_december_27st(input_folder, models, scenarios, 2100)

In [ ]:
#重要！检查原始数据是否缺失部分日期数据,2月29日

In [ ]:
import os
import subprocess

def check_for_february_29(input_folder, models, scenarios, year):
    """
    检查指定模型、情景和特定年份的NetCDF文件是否包含2月29日的数据。
    """
    for model in models:
        for scenario in scenarios:
            # 构建文件路径
            model_path = os.path.join(input_folder, model, scenario, "tas")

            # 根据模型和情景构建文件名
            if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
            elif model in ["FGOALS-g3"]:
                file_name = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
            elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
            elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
            elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
            else:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"

            file_path = os.path.join(model_path, file_name)
            wsl_file_path = file_path.replace("L:", "/mnt/l").replace("\\", "/")

            if os.path.exists(file_path):
                try:
                    # 使用CDO的showdate命令显示文件中的日期
                    result = subprocess.run(["wsl", "cdo", "showdate", wsl_file_path], 
                                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, check=True)
                    dates = result.stdout.strip().split()
                    february_29 = f"{year}-02-29"
                    if february_29 in dates:
                        print(f"February 29 is present in {file_path}")
                    else:
                        print(f"February 29 is missing in {file_path}")
                except subprocess.CalledProcessError as e:
                    print(f"Error processing file {file_path}: {e}")
            else:
                print(f"File not found: {file_path}")

# 输入文件夹的Windows路径
input_folder = "L:/GDDP-CMIP6"

# 模型列表
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 情景列表
scenarios = ["ssp126", "ssp245", "ssp370", "ssp585"]

# 只检查2020年的数据
check_for_february_29(input_folder, models, scenarios, 2020)

In [ ]:
#重要！这个是最终的
#KACE-1-0-G和UKESM1-0-LL两个模式的最长日期没有31日（1月、3月、5月、7月、8月、10月和12月都没有），导致计算有误，不要这两个模式，
#没有2月29日的原始数据模式包括BCC-CSM2-MR、CanESM5、CMCC-CM2-SR5、CMCC-ESM2、FGOALS-g3、GFDL-ESM4、GISS-E2-1-G、INM-CM4-8、INM-CM5-0、NorESM2-LM、NorESM2-MM、TaiESM1
#bcc-csm2-mr这个模式没有湿度数据，考虑到后面要用到湿度数据，也不要这个模式
#这个不能同一台电脑运行多个界面，否则速度将至不到三分之一

In [ ]:
import os
import subprocess
from tqdm import tqdm
from datetime import datetime

def is_date_present_in_file(wsl_file_path, date_to_check):
    """
    检查指定的WSL路径下的文件是否包含特定日期。
    使用 CDO showdate 命令来检查。
    """
    try:
        result = subprocess.run(
            ["wsl", "cdo", "showdate", wsl_file_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            check=True,
            encoding='utf-8'
        )
        dates = result.stdout.strip().split()
        return date_to_check in dates
    except subprocess.CalledProcessError:
        return False

def is_cdo_available():
    """
    检查CDO工具是否在WSL中可用。
    """
    try:
        subprocess.run(["wsl", "cdo", "-V"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

if not is_cdo_available():
    print("cdo is not installed or not found in PATH in WSL. Please install cdo in WSL and ensure it is in the PATH.")
    exit()

# 输入文件夹的Windows路径
input_folder = "L:/GDDP-CMIP6"

# 模型列表
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 情景和对应年份范围的字典
scenarios_years = {
    "ssp126": range(2015, 2101),
    "ssp245": range(2015, 2101),
    "ssp370": range(2015, 2101),
    "ssp585": range(2015, 2101)
}

# 特定日期缺失的模型字典
models_missing_dates = {
    "February_29": ["CanESM5", "CMCC-CM2-SR5", "CMCC-ESM2", "FGOALS-g3", "GFDL-ESM4", "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "NorESM2-LM", "NorESM2-MM", "TaiESM1"],
    "End_of_Month_31": ["KACE-1-0-G", "UKESM1-0-LL"]
}

# 遍历每个情景和其对应的年份范围
for scenario, years in scenarios_years.items():
    output_folder = f"L:/CMIP6-1final/{scenario}"

    # 创建输出文件夹（如果不存在）
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for year in tqdm(years):
        start_time = datetime.now()  # 开始时间
        files_to_process = []

        # 特定日期检查
        february_29 = f"{year}-02-29"
        months_with_31_days = [1, 3, 5, 7, 8, 10, 12]

        # 遍历所有模型
        for model in models:
            model_path = os.path.join(input_folder, model, scenario, "tas")
            
            # 构建完整的文件名和路径
            if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
            elif model in ["FGOALS-g3"]:
                file_name = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
            elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
            elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
            elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
            else:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"
                
            file_path = os.path.join(model_path, file_name)
            wsl_file_path = file_path.replace("L:", "/mnt/l").replace("\\", "/")

            if os.path.exists(file_path):
                include_file = True

                # 检查特定模型是否缺失2月29日
                if model in models_missing_dates["February_29"]:
                    if not is_date_present_in_file(wsl_file_path, february_29):
                        include_file = False

                # 检查特定模型是否缺失包含31日的月份
                if model in models_missing_dates["End_of_Month_31"]:
                    for month in months_with_31_days:
                        last_day = f"{year}-{month:02d}-31"
                        if not is_date_present_in_file(wsl_file_path, last_day):
                            include_file = False
                            break

                if include_file:
                    files_to_process.append(wsl_file_path)

        # 处理文件并计算均值
        if files_to_process:
            output_file = os.path.join(output_folder, f"tas_day_{scenario}_r1i1p1f1_gn_{year}.nc")
            wsl_output_file = output_file.replace("L:", "/mnt/l").replace("\\", "/")

            cdo_command = ["wsl", "cdo", "ensmean"] + files_to_process + [wsl_output_file]

            try:
                subprocess.run(cdo_command, check=True)
                print(f"Processed and saved data for scenario {scenario}, year {year}")
            except subprocess.CalledProcessError as e:
                print(f"Error processing data for scenario {scenario}, year {year}: {e}")

            end_time = datetime.now()  # 结束时间
            elapsed_time = end_time - start_time
            print(f"Time taken for scenario {scenario}, year {year}: {elapsed_time}")

        else:
            print(f"No data found for scenario {scenario}, year {year}. Skipping.")

In [ ]:
#KACE-1-0-G和UKESM1-0-LL两个模式的最长日期没有31日（1月、3月、5月、7月、8月、10月和12月都没有），导致计算有误，不要这两个模式，
#bcc-csm2-mr这个模式没有湿度数据，考虑到后面要用到湿度数据，也不要这个模式
#这个不能同一台电脑运行多个界面，否则速度将至不到三分之一

In [ ]:
import os
import subprocess
from tqdm import tqdm
from datetime import datetime

def is_cdo_available():
    """
    检查CDO是否在WSL中可用。
    """
    try:
        # 使用WSL运行CDO的版本检查命令
        subprocess.run(["wsl", "cdo", "-V"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

if not is_cdo_available():
    print("cdo is not installed or not found in PATH in WSL. Please install cdo in WSL and ensure it is in the PATH.")
    exit()

# 输入文件夹的Windows路径
input_folder = "L:/GDDP-CMIP6"

# 模型列表
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", "MPI-ESM1-2-LR", 
    "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", "TaiESM1",
]

# 情景和对应年份范围的字典
scenarios_years = {
    "ssp126": range(2015, 2101),
    "ssp245": range(2015, 2101),
    "ssp370": range(2015, 2101),
    "ssp585": range(2015, 2101)
}

# 遍历每个情景和其对应的年份范围
for scenario, years in scenarios_years.items():
    output_folder = f"L:/CMIP6-1new/{scenario}"  # 根据情景设置输出文件夹

    # 创建输出文件夹（如果不存在）
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for year in tqdm(years):
        start_time = datetime.now()  # 记录开始时间

        files_to_process = []

        # 遍历所有模型
        for model in models:
            model_path = os.path.join(input_folder, model, scenario, "tas")

            # 根据模型和情景构建文件名
            if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
            elif model in ["FGOALS-g3"]:
                file_name = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
            elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
            elif model in ["GISS-E2-1-G", "MIROC-ES2L"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
            elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
            else:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"
                
            file_path = os.path.join(model_path, file_name)  # 构建完整的文件路径

            # 将Windows路径转换为WSL路径
            wsl_file_path = file_path.replace("L:", "/mnt/l").replace("\\", "/")

            if os.path.exists(file_path):
                files_to_process.append(wsl_file_path)
            else:
                print(f"Warning: File not found {file_path}")

        # 如果有文件要处理
        if files_to_process:
            output_file = os.path.join(output_folder, f"tas_day_{scenario}_r1i1p1f1_gn_{year}.nc")
            wsl_output_file = output_file.replace("L:", "/mnt/l").replace("\\", "/")

            cdo_command = ["wsl", "cdo", "ensmean"] + files_to_process + [wsl_output_file]

            try:
                subprocess.run(cdo_command, check=True)
                print(f"Processed and saved data for scenario {scenario}, year {year}")
            except subprocess.CalledProcessError as e:
                print(f"Error processing data for scenario {scenario}, year {year}: {e}")

            end_time = datetime.now()  # 记录结束时间
            elapsed_time = end_time - start_time
            print(f"Time taken for scenario {scenario}, year {year}: {elapsed_time}")

        else:
            print(f"No data found for scenario {scenario}, year {year}. Skipping.")

In [ ]:
#计算26个模式每日均值，用的是CDO，每年大概需要4分钟

In [ ]:
import os
import subprocess
from tqdm import tqdm

def is_cdo_available():
    """
    检查CDO是否在WSL中可用。
    """
    try:
        # 使用WSL运行CDO的版本检查命令
        subprocess.run(["wsl", "cdo", "-V"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

if not is_cdo_available():
    print("cdo is not installed or not found in PATH in WSL. Please install cdo in WSL and ensure it is in the PATH.")
    exit()

# 输入文件夹的Windows路径
input_folder = "L:/GDDP-CMIP6"

# 模型列表
models = [
    "ACCESS-CM2", "ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", 
    "CMCC-CM2-SR5", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", 
    "EC-Earth3", "EC-Earth3-Veg-LR", "FGOALS-g3", "GFDL-ESM4", 
    "GISS-E2-1-G", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", 
    "KACE-1-0-G", "MIROC6", "MIROC-ES2L", "MPI-ESM1-2-HR", 
    "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "NorESM2-MM", 
    "TaiESM1", "UKESM1-0-LL"
]

# 情景和对应年份范围的字典
scenarios_years = {
    "ssp126": range(2015, 2101),  # 情景ssp126，年份为2032到2033年
    "ssp245": range(2015, 2101),  # 情景ssp245，年份为2067到2100年
    "ssp370": range(2015, 2101),  # 情景ssp370，年份为2015到2100年
    "ssp585": range(2015, 2101)   # 情景ssp585，年份为2015到2100年
}

# 遍历每个情景和其对应的年份范围
for scenario, years in scenarios_years.items():
    output_folder = f"L:/CMIP6-1/{scenario}"  # 根据情景设置输出文件夹

    # 创建输出文件夹（如果不存在）
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for year in tqdm(years):
        files_to_process = []

        # 遍历所有模型
        for model in models:
            model_path = os.path.join(input_folder, model, scenario, "tas")

            # 根据模型和情景构建文件名
            if model in ["CNRM-CM6-1", "CNRM-ESM2-1"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gr_{year}.nc"
            elif model in ["FGOALS-g3"]:
                file_name = f"tas_day_{model}_{scenario}_r3i1p1f1_gn_{year}.nc"
            elif model in ["GFDL-ESM4", "INM-CM4-8", "INM-CM5-0"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr1_{year}.nc"
            elif model in ["GISS-E2-1-G", "MIROC-ES2L", "UKESM1-0-LL"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f2_gn_{year}.nc"
            elif model in ["EC-Earth3", "EC-Earth3-Veg-LR", "IPSL-CM6A-LR", "KACE-1-0-G"]:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gr_{year}.nc"
            else:
                file_name = f"tas_day_{model}_{scenario}_r1i1p1f1_gn_{year}.nc"

            file_path = os.path.join(model_path, file_name)  # 构建完整的文件路径

            # 将Windows路径转换为WSL路径
            wsl_file_path = file_path.replace("L:", "/mnt/l").replace("\\", "/")

            if os.path.exists(file_path):
                files_to_process.append(wsl_file_path)
            else:
                print(f"Warning: File not found {file_path}")

        # 如果有文件要处理
        if files_to_process:
            output_file = os.path.join(output_folder, f"tas_day_{scenario}_r1i1p1f1_gn_{year}.nc")
            wsl_output_file = output_file.replace("L:", "/mnt/l").replace("\\", "/")

            cdo_command = ["wsl", "cdo", "ensmean"] + files_to_process + [wsl_output_file]

            try:
                subprocess.run(cdo_command, check=True)
                print(f"Processed and saved data for scenario {scenario}, year {year}")
            except subprocess.CalledProcessError as e:
                print(f"Error processing data for scenario {scenario}, year {year}: {e}")
        else:
            print(f"No data found for scenario {scenario}, year {year}. Skipping.")

In [ ]:
#计算完结果后，用下面代码检查下结果是否包含一年所有日期。

In [ ]:
import os
import subprocess

def check_for_december_31st(input_folder, scenarios, historical=False):
    """
    检查特定情景和年份的NetCDF文件是否包含12月31日的数据。
    """
    # 设置年份范围
    year_range = range(1990, 2015) if historical else range(2015, 2101)

    for scenario in scenarios:
        for year in year_range:
            # 构建文件路径
            if historical:
                file_name = f"tas_day_historical_r1i1p1f1_gn_{year}.nc"
                file_path = os.path.join(input_folder, "historical", file_name)
            else:
                file_name = f"tas_day_{scenario}_r1i1p1f1_gn_{year}.nc"
                file_path = os.path.join(input_folder, scenario, file_name)
            
            wsl_file_path = file_path.replace("L:", "/mnt/l").replace("\\", "/")

            if os.path.exists(file_path):
                try:
                    # 使用CDO的showdate命令显示文件中的日期
                    result = subprocess.run(["wsl", "cdo", "showdate", wsl_file_path], 
                                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, check=True)
                    dates = result.stdout.strip().split()
                    december_31st = f"{year}-12-31"
                    if december_31st in dates:
                        print(f"December 31st is present in {file_path}")
                    else:
                        print(f"December 31st is missing in {file_path}")
                except subprocess.CalledProcessError as e:
                    print(f"Error processing file {file_path}: {e}")
            else:
                print(f"File not found: {file_path}")

# 输入文件夹的Windows路径
input_folder = "L:/CMIP6-1new"

# 情景列表
scenarios = ["ssp126", "ssp245", "ssp370", "ssp585"]

# 检查2025年的数据
check_for_december_31st(input_folder, scenarios)

# 检查历史数据
check_for_december_31st(input_folder, scenarios, historical=True)

In [ ]:
#重要！上述的代码计算了25个模式的均值，如下代码则基于湿度和温度数据计算人体热指数

In [ ]:
import xarray as xr
import numpy as np
import datetime
import os

# 定义情景和年份
scenarios = ['ssp126', 'ssp245','ssp370', 'ssp585']
years = range(2015, 2101)

#scenarios = ['historical']
#years = range(1990, 2015)

for scenario in scenarios:
    for year in years:
        # 构建文件路径
        temperature_file = f'L:/CMIP6-1final/{scenario}/tas_day_{scenario}_r1i1p1f1_gn_{year}.nc'
        humidity_file = f'M:/CMIP6-1final/{scenario}/hurs_day_{scenario}_r1i1p1f1_gn_{year}.nc'
        output_file = f'L:/CMIP6-1NEW/{scenario}/hi_day_{scenario}_r1i1p1f1_gn_{year}.nc'

        # 检查并创建输出目录
        output_directory = os.path.dirname(output_file)
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        # 检查输出文件是否存在
        if not os.path.exists(output_file):
            # 加载数据
            temp_data = xr.open_dataset(temperature_file)
            humidity_data = xr.open_dataset(humidity_file)

            # 获取温度和相对湿度的数据
            T = temp_data['tas']  # 'tas' 变量为温度
            RH = humidity_data['hurs']  # 'hurs' 变量为湿度

            # 计算热指数（HI）
            T_celsius = T - 273.15
            HI = xr.where((T <= 299.81667) | (RH <= 40), T_celsius, 
                          -8.78469475556 + 1.61139411 * T_celsius + 2.33854883889 * RH 
                          - 0.14611605 * T_celsius * RH - 0.012308094 * (T_celsius ** 2) 
                          - 0.0164248277778 * (RH ** 2) + 0.002211732 * (T_celsius ** 2) * RH 
                          + 0.00072546 * T_celsius * (RH ** 2) - 0.000003582 * (T_celsius ** 2) * (RH ** 2))

            # 保存结果为新的NetCDF文件
            HI.to_netcdf(output_file)

            # 显示当前时间
            print(f"已完成 {scenario} 情景 {year} 年的数据，当前时间: {datetime.datetime.now()}")
        else:
            print(f"文件 {output_file} 已存在，跳过此文件。")

print("所有热指数计算完成。")

In [ ]:
#对0.25°*0.25°原始数据进行降尺度处理，转变为0.125°*0.125°数据，但需要注意的是新生成数据的extent为top为90.0625，left为-0.0625, right为360.0625, bottom为-60.0625
#原始数据的extent 为top为90，left为0，right为360，bottom为-60

In [ ]:
import xarray as xr
import numpy as np
import os

# 定义年份和情景
years = [1990] + list(range(2000, 2020)) + list(range(2020, 2101, 10))
scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

# 定义原始数据和重采样数据的文件夹路径
input_folder = 'L:/CMIP6-1NEW0.25'
output_folder = 'L:/CMIP6-1NEW0.125'

# 创建输出文件夹（如果不存在）
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)
    print(f"创建文件夹：{output_folder}")

for scenario in scenarios:
    scenario_output_folder = os.path.join(output_folder, scenario)
    if not os.path.exists(scenario_output_folder):
        os.makedirs(scenario_output_folder, exist_ok=True)
        print(f"创建文件夹：{scenario_output_folder}")

    for year in years:
        input_file = f'{input_folder}/{scenario}/hi_day_{scenario}_r1i1p1f1_gn_{year}.nc'
        output_file = f'{scenario_output_folder}/hi_day_{scenario}_r1i1p1f1_gn_{year}_0.125.nc'

        # 确保输入文件存在
        if os.path.exists(input_file):
            ds = xr.open_dataset(input_file)

            # 定义新的纬度和经度，确保边界与原始数据对齐
            new_lat = np.arange(-60, 90.125, 0.125)  # 修改范围，包括90度
            new_lon = np.arange(0, 360.125, 0.125)   # 修改范围，包括360度

            # 确保包含边界
            if new_lat[-1] > 90:
                new_lat = new_lat[:-1]
            if new_lon[-1] > 360:
                new_lon = new_lon[:-1]

            # 使用最近邻插值进行重采样
            ds_resampled = ds.interp(lat=new_lat, lon=new_lon, method='nearest')

            # 保留原始数据集的属性
            ds_resampled.attrs = ds.attrs
            ds_resampled.lat.attrs = ds.lat.attrs
            ds_resampled.lon.attrs = ds.lon.attrs

            # 保存重采样后的数据集
            ds_resampled.to_netcdf(output_file)

            print(f'Resampled {scenario} for year {year}')
        else:
            print(f"输入文件不存在：{input_file}")

In [ ]:
#重要！计算全球气温阈值

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import netCDF4 as nc
from datetime import datetime, timedelta
import calendar
import time  # 引入 time 模块

def calculate_threshold(data, threshold=18.3, increment=0.2):
    nodata_mask = data.isnull().all(dim='time')
    count_above_threshold = (data > threshold).sum(dim='time')
    mean_temp = data.where(data > threshold).mean(dim='time')
    new_threshold = xr.where(count_above_threshold >= 3, threshold + increment * (mean_temp - threshold), threshold)
    new_threshold = xr.where(nodata_mask, np.nan, new_threshold)
    return new_threshold

def process_data(start_year=2000, end_year=2100, step=5, scenarios=['ssp126']):
    for scenario in scenarios:
        for year in range(start_year, end_year + 1, step):
            process_year_data(year, scenario)
            
def process_year_data(year, scenario):
    start_time = time.time()  # 记录处理开始时间
    input_directory = f'L:/CMIP6-1NEW/{scenario}/'
    output_directory = f'L:/CMIP6-1yuzhi/{scenario}/'

    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)

    output_file_path = f'{output_directory}hi_yuzhi_day_{scenario}_r1i1p1f1_gn_{year}.nc'
    all_dates = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31")

    # 根据2019年数据的分辨率设置纬度和经度大小
    lat_size = 600  # 纬度大小
    lon_size = 1440  # 经度大小

    # 初始化年度数据集
    with nc.Dataset(output_file_path, 'w', format='NETCDF4') as ds:
        # 创建时间维度
        ds.createDimension('time', None)
        time_var = ds.createVariable('time', 'f8', ('time',))
        time_var.units = 'hours since 1800-01-01'
        time_var.calendar = 'gregorian'
        time_var.standard_name = 'time'
        time_var.long_name = 'time'
        time_var.axis = 'T'

        # 创建纬度和经度维度
        ds.createDimension('lat', lat_size)
        ds.createDimension('lon', lon_size)
        lat_var = ds.createVariable('lat', 'f4', ('lat',))
        lon_var = ds.createVariable('lon', 'f4', ('lon',))
        lat_var[:] = np.linspace(-59.875, 89.875, lat_size)
        lon_var[:] = np.linspace(0.125, 359.875, lon_size)

        # 设置纬度和经度的坐标系统属性
        lat_var.standard_name = 'latitude'
        lat_var.long_name = 'latitude'
        lat_var.units = 'degrees_north'
        lat_var.axis = 'Y'
        lat_var.crs = 'EPSG:4326'

        lon_var.standard_name = 'longitude'
        lon_var.long_name = 'longitude'
        lon_var.units = 'degrees_east'
        lon_var.axis = 'X'
        lon_var.crs = 'EPSG:4326'

        # 创建数据变量
        data_var = ds.createVariable('__xarray_dataarray_variable__', 'f4', ('time', 'lat', 'lon'), fill_value=np.nan)

    # 每日数据处理
    for date in all_dates:
        month = date.month
        day = date.day
        past_data = []
        for past_year in range(year - 5, year):
            file_path = f'{input_directory}hi_day_{scenario}_r1i1p1f1_gn_{past_year}.nc'  # 修改为动态情景名称
            if os.path.exists(file_path):
                with xr.open_dataset(file_path) as ds:
                    past_date_str = date.strftime(f"{past_year}-%m-%d")
                    if (month, day) == (2, 29) and not calendar.isleap(past_year):
                        past_date_str = f"{past_year}-02-28"
                    ds = ds.sel(time=past_date_str)
                    past_data.append(ds)

        if past_data:
            combined_data = xr.concat(past_data, dim='time')
            threshold_data = calculate_threshold(combined_data['__xarray_dataarray_variable__'])
            
            with nc.Dataset(output_file_path, 'a') as dst:
                time_index = len(dst['time'])
                # 为了与2019年数据对齐，将时间设置为中午12点
                dst['time'][time_index] = nc.date2num(date.replace(hour=12), dst['time'].units, dst['time'].calendar)
                dst['__xarray_dataarray_variable__'][time_index, :, :] = threshold_data.values

    end_time = time.time()  # 记录处理结束时间
    elapsed_time = end_time - start_time  # 计算所用时间
    print(f'年度数据集 {output_file_path} 已成功保存。用时: {elapsed_time:.2f} 秒')

process_data()

In [ ]:
#对0.25°*0.25°原始阈值数据进行降尺度处理，转变为0.125°*0.125°数据，但需要注意的是新生成数据的extent为top为90.0625，left为-0.0625, right为360.0625, bottom为-60.0625
#原始数据的extent 为top为90，left为0，right为360，bottom为-60

In [ ]:
import xarray as xr
import numpy as np
import os

# 定义数据文件名
filenames = ['hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.1.nc', 'hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.2.nc']

# 定义原始数据和重采样数据的文件夹路径
input_folder = 'L:/CMIP6-1yuzhi0.25'
output_folder = 'L:/CMIP6-1yuzhi0.125'

# 创建输出文件夹（如果不存在）
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)
    print(f"创建文件夹：{output_folder}")

# 循环处理每个文件
for filename in filenames:
    input_file = os.path.join(input_folder, filename)
    output_file = os.path.join(output_folder, filename.replace('.nc', '_0.125.nc'))

    # 确保输入文件存在
    if os.path.exists(input_file):
        ds = xr.open_dataset(input_file)

        # 定义新的纬度和经度，确保边界与原始数据对齐
        new_lat = np.arange(-60, 90.125, 0.125)  # 修改范围，包括90度
        new_lon = np.arange(0, 360.125, 0.125)   # 修改范围，包括360度

        # 确保包含边界
        if new_lat[-1] > 90:
            new_lat = new_lat[:-1]
        if new_lon[-1] > 360:
            new_lon = new_lon[:-1]

        # 使用最近邻插值进行重采样
        ds_resampled = ds.interp(lat=new_lat, lon=new_lon, method='nearest')

        # 保留原始数据集的属性
        ds_resampled.attrs = ds.attrs
        ds_resampled.lat.attrs = ds.lat.attrs
        ds_resampled.lon.attrs = ds.lon.attrs

        # 保存重采样后的数据集
        ds_resampled.to_netcdf(output_file)

        print(f'Resampled and saved: {output_file}')
    else:
        print(f"输入文件不存在：{input_file}")

In [ ]:
#将历史数据另存到各个情景文件夹中

In [ ]:
import os
import shutil

# 定义源目录和目标目录
source_directory = 'L:/CMIP6-1NEW/historical/'
target_directory = 'L:/CMIP6-1NEW/ssp585/'

# 定义需要处理的年份范围
years = range(1995, 2015)

# 遍历每个年份，复制并重命名文件
for year in years:
    source_file = f'{source_directory}hi_day_historical_r1i1p1f1_gn_{year}.nc'
    target_file = f'{target_directory}hi_day_ssp585_r1i1p1f1_gn_{year}.nc'
    
    # 检查源文件是否存在
    if os.path.exists(source_file):
        # 复制并重命名文件
        shutil.copyfile(source_file, target_file)
        print(f'文件 {year} 已复制和重命名。')
    else:
        print(f'源文件 {year} 不存在，跳过。')

print('所有文件处理完成。')

In [ ]:
#重点！合并日度数据为年度数据时一定要用netCDF4库！用其他的都容易出错
#考虑到“温度越高的地方，居民往往越耐热”。基于此，我们基于历史前五年数据计算该年的温度阈值，
#以栅格a为例，其在1月1日温度阈值假设为Ta0101。如果该栅格在2020年前五年（2015-2019年）的1月1日中没有3个以上年份的温度高于20摄氏度，那2020年1月1日温度阈值Ta0101为20摄氏度；如果栅格a在2020年前五年（2015-2019年）1月1日中存在3个以上年份的温度高于20摄氏度，那先计算栅格a这五年中高于20摄氏度的温度的均值Ta0101mean，而该栅格a在2020年1月1日的温度阈值Ta0101则为20+0.1*（Ta0101mean-20）。其他栅格的计算依次类推。直至计算出该年所有栅格的温度阈值。
#闰年（比如2020年）是有2月29日，但其前面五年（比如2015-2019年）中很多年份都没有2月29日，为此，闰年（比如2020年）所有栅格2月29日的温度阈值直接等同于该年对应栅格2月28日的温度阈值。

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import netCDF4 as nc
from datetime import datetime, timedelta
import calendar

def calculate_threshold(data, threshold=20, increment=0.1):
    nodata_mask = data.isnull().all(dim='time')
    count_above_threshold = (data > threshold).sum(dim='time')
    mean_temp = data.where(data > threshold).mean(dim='time')
    new_threshold = xr.where(count_above_threshold >= 3, threshold + increment * (mean_temp - threshold), threshold)
    new_threshold = xr.where(nodata_mask, np.nan, new_threshold)
    return new_threshold


def process_2020_data():
    input_directory = 'L:/CMIP6-1NEW/ssp126/'
    output_directory = 'L:/CMIP6-1yuzhi12bf/ssp126/'
    year = 2020

    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)

    output_file_path = f'{output_directory}hi_yuzhi_day_ssp126_r1i1p1f1_gn_{year}.nc'
    all_dates = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31")

    # 根据2019年数据的分辨率设置纬度和经度大小
    lat_size = 600  # 纬度大小
    lon_size = 1440  # 经度大小

    # 初始化年度数据集
    with nc.Dataset(output_file_path, 'w', format='NETCDF4') as ds:
        # 创建时间维度
        ds.createDimension('time', None)
        time_var = ds.createVariable('time', 'f8', ('time',))
        time_var.units = 'hours since 1800-01-01'
        time_var.calendar = 'gregorian'
        time_var.standard_name = 'time'
        time_var.long_name = 'time'
        time_var.axis = 'T'

        # 创建纬度和经度维度
        ds.createDimension('lat', lat_size)
        ds.createDimension('lon', lon_size)
        lat_var = ds.createVariable('lat', 'f4', ('lat',))
        lon_var = ds.createVariable('lon', 'f4', ('lon',))
        lat_var[:] = np.linspace(-59.875, 89.875, lat_size)  # 根据2019年数据调整
        lon_var[:] = np.linspace(0.125, 359.875, lon_size)  # 根据2019年数据调整

        # 创建数据变量
        data_var = ds.createVariable('__xarray_dataarray_variable__', 'f4', ('time', 'lat', 'lon'), fill_value=np.nan)

    # 每日数据处理
    for date in all_dates:
        month = date.month
        day = date.day
        past_data = []
        for past_year in range(year - 5, year):
            file_path = f'{input_directory}hi_day_ssp126_r1i1p1f1_gn_{past_year}.nc'
            if os.path.exists(file_path):
                with xr.open_dataset(file_path) as ds:
                    past_date_str = date.strftime(f"{past_year}-%m-%d")
                    if (month, day) == (2, 29) and not calendar.isleap(past_year):
                        past_date_str = f"{past_year}-02-28"
                    ds = ds.sel(time=past_date_str)
                    past_data.append(ds)

        if past_data:
            combined_data = xr.concat(past_data, dim='time')
            threshold_data = calculate_threshold(combined_data['__xarray_dataarray_variable__'])
            
            with nc.Dataset(output_file_path, 'a') as dst:
                time_index = len(dst['time'])
                # 为了与2019年数据对齐，将时间设置为中午12点
                dst['time'][time_index] = nc.date2num(date.replace(hour=12), dst['time'].units, dst['time'].calendar)
                dst['__xarray_dataarray_variable__'][time_index, :, :] = threshold_data.values

    print(f'年度数据集 {output_file_path} 已成功保存。')

process_2020_data()

In [ ]:
#1、计算每日的气温阈值数据都需要读取1950-1999年的750天的气温数据，是15天乘以50年。采用的方法是读取1950~1999年期间同一天的15天窗口。比如计算2000年1月1日的气温阈值需要读取的是1950-1999年1月1日至1月15日，计算1月2日的气温阈值需要读取的是1950-1999年1月1日至1月15日，计算1月3日的气温阈值需要读取的是1950-1999年1月1日至1月15日，依次类推，而计算1月8日的气温阈值需要读取的是1950-1999年1月1日至1月15日，计算1月9日的气温阈值需要读取的是1950-1999年1月2日至1月16日，计算1月10日的气温阈值需要读取的是1950-1999年1月3日至1月17日，依次类推，计算1月26日的气温阈值需要读取的是1950-1999年1月19日至2月2日，计算1月27日的气温阈值需要读取的是1950-1999年1月20日至2月3日，依次类推，计算2月21日的气温阈值需要读取的是1950-1999年2月14日至2月28日，计算2月22日的气温阈值需要读取的是1950-1999年2月15日至3月1日（闰年2月29日不考虑在内），计算2月23日的气温阈值需要读取的是1950-1999年2月16日至3月2日（闰年2月29日不考虑在内），依次类推，计算2月28日的气温阈值需要读取的是1950-1999年2月21日至3月7日（闰年2月29日不考虑在内），而计算2月29日的气温阈值时则直接等于前面计算出的2月28日的气温阈值，计算3月1日的气温阈值需要读取的是1950-1999年2月22日至3月8日（闰年2月29日不考虑在内），计算3月2日的气温阈值需要读取的是1950-1999年2月23日至3月9日（闰年2月29日不考虑在内），依次类推，计算3月8日的气温阈值需要读取的是1950-1999年3月1日至3月15日，依次类推，计算12月29日的气温阈值需要读取的是1950-1999年12月17日至12月31日，计算12月30日的气温阈值需要读取的是1950-1999年12月17日至12月31日，计算12月28日的气温阈值需要读取的是1950-1999年12月17日至12月31日。
#2、以栅格a为例，其在2000年1月1日温度阈值假设为Ta0101。如果该栅格在该日1950-1999年对应的150天中没有60%的天数的温度高于18.3，那该栅格2000年1月1日温度阈值Ta0101为18.3摄氏度；如果该栅格在该日1950-1999年对应的150天中有60%的天数的温度高于18.3，那先计算栅格a这150天中高于18.3摄氏度的温度的均值Ta0101mean，而该栅格a在2000年1月1日的温度阈值Ta0101则为18.3+0.1*（Ta0101mean-18.3）。2000年是闰年，是有2月29日的，该日温度阈值直接用2月28日的。依次计算出栅格b阈值数据的情况，直至计算出所有栅格的阈值数据。
#3、生成的数据文件为L:\CMIP6-1yuzhi\hi_yuzhi_day_r1i1p1f1_gn_2000.nc, 空间精度为0.25度*0.25度，日期是1月1日-12月31日，日期格式类似于为2000/01/01 12:00:00 AM，2000/01/02 12:00:00 AM，2000/01/03 12:00:00 AM直至2000/12/31 12:00:00 AM。该数据文件的raster information 中columns and rows 是1440,600。number of bands 是1。cell size是0.25°*0.25°。Format是NetCDF。Source type是generic。Pixel type是floating point。pixel Depth是32 Bit。NoData Value 是nan。Status是Permanent。Extent的top是90，bottom是-60，left是0，right是360。XY Coordinate System是GCS_WGS_1984。Angular Unit是Degree（0.0174532925199433）。Datum是D_WGS_1984。
#4、当读取1950-1999年栅格a某个日期的数据为nodata，最后生成的阈值数据L:\CMIP6-1yuzhi\hi_yuzhi_day _r1i1p1f1_gn_2000.nc该栅格全年也应该是nodata，且不显示。参考这段代码处理方式nodata_mask = data.isnull().all(dim='time')

In [ ]:
import xarray as xr
import numpy as np
import os
import netCDF4 as nc
import pandas as pd
import calendar
import time

def calculate_threshold(data, threshold=18.3, percentage=0.6, increment=0.1):
    # 检查每个栅格在整个时间序列中是否全部为nodata
    nodata_mask = data.isnull().all(dim='time')

    # 只在非nodata的数据上计算阈值
    valid_data_mask = ~nodata_mask

    count_above_threshold = (data > threshold).where(valid_data_mask).sum(dim='time')
    mean_temp_above_threshold = data.where(data > threshold).where(valid_data_mask).mean(dim='time')

    percentage_above_threshold = count_above_threshold / len(data.time)
    adjusted_threshold = threshold + increment * (mean_temp_above_threshold - threshold)
    new_threshold = xr.where(percentage_above_threshold >= percentage, adjusted_threshold, threshold)
    
    new_threshold = xr.where(nodata_mask, np.nan, new_threshold)  # 保留nodata的位置

    return new_threshold

def get_date_ranges(date):
    year = date.year
    month = date.month
    day = date.day

    # 确定每个月的天数
    _, num_days_in_month = calendar.monthrange(year, month)

    # 计算15天窗口的开始和结束日期
    if month == 1 and day < 8:
        start_day = 1
        end_day = 15
    elif month == 12 and day > 22:
        start_day = 18
        end_day = 31
    else:
        start_day = max(1, day - 7)
        end_day = min(num_days_in_month, day + 7)

    # 如果是2月并且天数超过了28，对于非闰年，调整天数
    if month == 2 and end_day > 28 and not calendar.isleap(year):
        end_day = 28

    # 生成开始和结束日期
    start_date = pd.Timestamp(year=year, month=month, day=start_day, hour=12)
    end_date = pd.Timestamp(year=year, month=month, day=end_day, hour=12)

    return start_date.isoformat(), end_date.isoformat()

def process_2000_data():
    input_directory = 'L:/CMIP6-1NEW/historical/'
    output_directory = 'L:/CMIP6-1yuzhi/'
    year = 2000

    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)

    output_file_path = f'{output_directory}hi_yuzhi_day_ssp126_r1i1p1f1_gn_{year}.nc'
    all_dates = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31")

    # 根据2019年数据的分辨率设置纬度和经度大小
    lat_size = 600  # 纬度大小
    lon_size = 1440  # 经度大小

    # 初始化年度数据集
    with nc.Dataset(output_file_path, 'w', format='NETCDF4') as ds:
        # 创建时间维度
        ds.createDimension('time', None)
        time_var = ds.createVariable('time', 'f8', ('time',))
        time_var.units = 'hours since 1800-01-01'
        time_var.calendar = 'gregorian'
        time_var.standard_name = 'time'
        time_var.long_name = 'time'
        time_var.axis = 'T'

        # 创建纬度和经度维度
        ds.createDimension('lat', lat_size)
        ds.createDimension('lon', lon_size)
        lat_var = ds.createVariable('lat', 'f4', ('lat',))
        lon_var = ds.createVariable('lon', 'f4', ('lon',))
        lat_var[:] = np.linspace(-59.875, 89.875, lat_size)  # 根据2019年数据调整
        lon_var[:] = np.linspace(0.125, 359.875, lon_size)  # 根据2019年数据调整
        lat_var.standard_name = 'latitude'
        lat_var.long_name = 'latitude'
        lat_var.units = 'degrees_north'
        lon_var.standard_name = 'longitude'
        lon_var.long_name = 'longitude'
        lon_var.units = 'degrees_east'

        # 创建数据变量
        data_var = ds.createVariable('__xarray_dataarray_variable__', 'f4', ('time', 'lat', 'lon'), fill_value=np.nan)

        # 添加描述性的全局属性
        ds.title = 'Climate Data'
        ds.institution = 'Your Institution'
        ds.source = 'Climate Model'
        ds.references = 'Your References'
        ds.comment = 'Processed using netCDF4 Python module'
        ds.geospatial_lat_min = -59.875
        ds.geospatial_lat_max = 89.875
        ds.geospatial_lon_min = 0.125
        ds.geospatial_lon_max = 359.875
        ds.geospatial_lat_units = "degrees_north"
        ds.geospatial_lon_units = "degrees_east"
        
       
    # 储存上一天的阈值数据，初始为None
    previous_day_threshold = None
    
    for date in all_dates:
        start_time = time.time()

        # 特别处理闰年的2月29日
        if date.month == 2 and date.day == 29:
            threshold_data = previous_day_threshold
        else:
            past_data = []
            for past_year in range(1950, 2000):
                file_path = f'{input_directory}hi_day_historical_r1i1p1f1_gn_{past_year}.nc'
                if os.path.exists(file_path):
                    with xr.open_dataset(file_path) as ds:
                        start_date, end_date = get_date_ranges(pd.Timestamp(year=past_year, month=date.month, day=date.day, hour=12))
                        ds_range = ds.sel(time=slice(start_date, end_date))
                        if ds_range['__xarray_dataarray_variable__'].size > 0:
                            past_data.append(ds_range['__xarray_dataarray_variable__'])

            if past_data:
                combined_data = xr.concat(past_data, dim='time')
                threshold_data = calculate_threshold(combined_data)

            # 存储当前日期的阈值数据以用于下一次循环（如2月29日）
            previous_day_threshold = threshold_data

            with nc.Dataset(output_file_path, 'a') as dst:
                time_index = len(dst['time'])
                dst['time'][time_index] = nc.date2num(date.replace(hour=12), dst['time'].units, dst['time'].calendar)
                dst['__xarray_dataarray_variable__'][time_index, :, :] = threshold_data.values

        end_time = time.time()
        print(f"Completed date: {date}, Time taken: {end_time - start_time} seconds")

    print(f'年度数据集 {output_file_path} 已成功保存。')

process_2000_data()

In [ ]:
#1、	以栅格a为例，其在1月温度阈值假设为Ta01。如果该栅格在1950-1999年的所有1月中没有60%的日期的温度高于18.3摄氏度，那该栅格a在1月温度阈值Ta01为18.3摄氏度；如果栅格a在1950-1999年1月中存在60%以上日期的温度高于18.3摄氏度，那先计算这些高于18.3摄氏度的日期的气温均值Tmean，而该栅格a在1月的气温阈值Ta01则为18.3+0.2*（Tmean-18.3）。其他栅格的计算依次类推。需要注意的是1月需要读取的日期是1950年-1999年的1月所有日期，为50*31=1550天。

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import netCDF4 as nc
from datetime import datetime, timedelta
import calendar

def calculate_threshold(data, threshold=20, increment=0.1):
    nodata_mask = data.isnull().all(dim='time')
    count_above_threshold = (data > threshold).sum(dim='time')
    mean_temp = data.where(data > threshold).mean(dim='time')
    new_threshold = xr.where(count_above_threshold >= 3, threshold + increment * (mean_temp - threshold), threshold)
    new_threshold = xr.where(nodata_mask, np.nan, new_threshold)
    return new_threshold


def process_2020_data():
    input_directory = 'L:/CMIP6-1NEW/ssp126/'
    output_directory = 'L:/CMIP6-1yuzhi12bf/ssp126/'
    year = 2020

    if not os.path.exists(output_directory):
        os.makedirs(output_directory, exist_ok=True)

    output_file_path = f'{output_directory}hi_yuzhi_day_ssp126_r1i1p1f1_gn_{year}.nc'
    all_dates = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31")

    # 根据2019年数据的分辨率设置纬度和经度大小
    lat_size = 600  # 纬度大小
    lon_size = 1440  # 经度大小

    # 初始化年度数据集
    with nc.Dataset(output_file_path, 'w', format='NETCDF4') as ds:
        # 创建时间维度
        ds.createDimension('time', None)
        time_var = ds.createVariable('time', 'f8', ('time',))
        time_var.units = 'hours since 1800-01-01'
        time_var.calendar = 'gregorian'
        time_var.standard_name = 'time'
        time_var.long_name = 'time'
        time_var.axis = 'T'

        # 创建纬度和经度维度
        ds.createDimension('lat', lat_size)
        ds.createDimension('lon', lon_size)
        lat_var = ds.createVariable('lat', 'f4', ('lat',))
        lon_var = ds.createVariable('lon', 'f4', ('lon',))
        lat_var[:] = np.linspace(-59.875, 89.875, lat_size)  # 根据2019年数据调整
        lon_var[:] = np.linspace(0.125, 359.875, lon_size)  # 根据2019年数据调整

        # 创建数据变量
        data_var = ds.createVariable('__xarray_dataarray_variable__', 'f4', ('time', 'lat', 'lon'), fill_value=np.nan)

    # 每日数据处理
    for date in all_dates:
        month = date.month
        day = date.day
        past_data = []
        for past_year in range(year - 5, year):
            file_path = f'{input_directory}hi_day_ssp126_r1i1p1f1_gn_{past_year}.nc'
            if os.path.exists(file_path):
                with xr.open_dataset(file_path) as ds:
                    past_date_str = date.strftime(f"{past_year}-%m-%d")
                    if (month, day) == (2, 29) and not calendar.isleap(past_year):
                        past_date_str = f"{past_year}-02-28"
                    ds = ds.sel(time=past_date_str)
                    past_data.append(ds)

        if past_data:
            combined_data = xr.concat(past_data, dim='time')
            threshold_data = calculate_threshold(combined_data['__xarray_dataarray_variable__'])
            
            with nc.Dataset(output_file_path, 'a') as dst:
                time_index = len(dst['time'])
                # 为了与2019年数据对齐，将时间设置为中午12点
                dst['time'][time_index] = nc.date2num(date.replace(hour=12), dst['time'].units, dst['time'].calendar)
                dst['__xarray_dataarray_variable__'][time_index, :, :] = threshold_data.values

    print(f'年度数据集 {output_file_path} 已成功保存。')

process_2020_data()

In [ ]:
#将L:\CMIP6-1NEW\historical\hi_day_historical_r1i1p1f1_gn_1990.nc保存为L:\CMIP6-1NEW\ssp126\hi_day_ssp126_r1i1p1f1_gn_1990.nc
#年份包括1990-1995年，情景包括ssp126，ssp245，ssp370，ssp585

In [ ]:
import os
import shutil

def copy_and_rename_files(start_year=1990, end_year=1995, scenarios=['ssp126', 'ssp245', 'ssp370', 'ssp585']):
    source_directory = 'L:/CMIP6-1NEW/historical/'
    target_base_directory = 'L:/CMIP6-1NEW/'

    for year in range(start_year, end_year + 1):
        source_file = f'{source_directory}hi_day_historical_r1i1p1f1_gn_{year}.nc'
        
        for scenario in scenarios:
            target_directory = f'{target_base_directory}{scenario}/'
            target_file = f'{target_directory}hi_day_{scenario}_r1i1p1f1_gn_{year}.nc'

            # 创建目标文件夹如果它不存在
            os.makedirs(target_directory, exist_ok=True)

            # 复制和重命名文件
            shutil.copy2(source_file, target_file)
            print(f'File copied: {source_file} to {target_file}')

copy_and_rename_files()

In [ ]:
#对2010-2100年进行重采样处理，原来数据的空间精度为0.125度*0.125度，重采样后的数据精度为0.25度*0.25度
#考虑到上述重采样得到的是均值，但是我们需要的是求和，所以我们采用先计算人口密度，再重采样，再求人口数量的方法
#这步是计算人口密度的代码

In [ ]:
import rasterio
import numpy as np
import os

def calculate_population_density(input_file, output_file):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file) as dataset:
        # 获取原始数据的 nodata 值
        nodata = dataset.nodata

        # 读取原始人口数据
        population_data = dataset.read(1)

        # 创建一个与人口数据相同形状的数组，用于存储每个像素的面积
        pixel_area = np.zeros(population_data.shape)

        # 计算每个像素的面积（单位：平方公里）
        for i in range(pixel_area.shape[0]):
            # 获取该行像素的平均纬度
            lat = dataset.xy(i, 0)[1]
            # 每个像素的高度（公里）
            pixel_height_km = 111.32 * 0.125
            # 每个像素的宽度（公里）
            pixel_width_km = 111.32 * np.cos(np.deg2rad(lat)) * 0.125
            # 计算面积
            pixel_area[i, :] = pixel_height_km * pixel_width_km

        # 计算人口密度：人口数 / 像素面积
        population_density = np.where(pixel_area > 0, population_data / pixel_area, nodata)

        # 确保原始数据中标记为Nodata的像素在结果中同样标记为Nodata
        population_density[population_data == nodata] = nodata

        # 将人口密度数据类型转换为 float32
        population_density = population_density.astype('float32')

        # 保存人口密度数据
        with rasterio.open(output_file, 'w', driver='GTiff', height=dataset.height,
                           width=dataset.width, count=1,
                           dtype='float32', crs=dataset.crs, transform=dataset.transform, nodata=nodata) as dst:
            dst.write(population_density, 1)

    print(f"Population density file saved: {output_file}")

# 新的处理逻辑
years = range(2010, 2101, 10)
ssps = ['ssp1', 'ssp2', 'ssp3', 'ssp5']
input_base_path = "L:/pop/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"
output_base_path = "L:/popden/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"

# 对每个SSP和年份的数据计算人口密度
for ssp in ssps:
    for year in years:
        input_file = f"{input_base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}.tif"
        output_file = f"{output_base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}den.tif"
        calculate_population_density(input_file, output_file)
        print(f"Population density calculated for {ssp}, year: {year}")

print("All population density calculations are completed.")

In [ ]:
#计算历史数据1990年人口密度，原数据精度为0.008333度*0.008333度

In [ ]:
import rasterio
import numpy as np
import os

def calculate_population_density_single(input_file, output_file):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file) as dataset:
        # 获取原始数据的 nodata 值
        nodata = dataset.nodata

        # 读取原始人口数据
        population_data = dataset.read(1)

        # 创建一个与人口数据相同形状的数组，用于存储每个像素的面积
        pixel_area = np.zeros(population_data.shape)

        # 计算每个像素的面积（单位：平方公里）
        for i in range(pixel_area.shape[0]):
            # 获取该行像素的平均纬度
            lat = dataset.xy(i, 0)[1]
            # 每个像素的高度（公里）
            pixel_height_km = 111.32 * 0.008333
            # 每个像素的宽度（公里）
            pixel_width_km = 111.32 * np.cos(np.deg2rad(lat)) * 0.008333
            # 计算面积
            pixel_area[i, :] = pixel_height_km * pixel_width_km

        # 计算人口密度：人口数 / 像素面积
        population_density = np.where(pixel_area > 0, population_data / pixel_area, nodata)

        # 确保原始数据中标记为Nodata的像素在结果中同样标记为Nodata
        population_density[population_data == nodata] = nodata

        # 将人口密度数据类型转换为 float32
        population_density = population_density.astype('float32')

        # 保存人口密度数据
        with rasterio.open(output_file, 'w', driver='GTiff', height=dataset.height,
                           width=dataset.width, count=1,
                           dtype='float32', crs=dataset.crs, transform=dataset.transform, nodata=nodata) as dst:
            dst.write(population_density, 1)

    print(f"Population density file saved: {output_file}")

# 设定输入和输出文件路径
input_file_path = "L:/pop/popdynamics-global-pop-count-time-series-estimates-1990-geotiff/popdynamics-global-pop-count-time-series-estimates_1990.tif"
output_file_path = "L:/popden/popdynamics-global-pop-count-time-series-estimates-1990-geotiff/popdynamics-global-pop-count-time-series-estimates_1990den.tif"

# 计算人口密度
calculate_population_density_single(input_file_path, output_file_path)

In [ ]:
#对未来人口密度数据进行重采样

In [ ]:
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_origin
import os

def resample_tif(input_file, output_file, desired_cell_size):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file) as dataset:
        # 获取原始数据的 nodata 值
        nodata = dataset.nodata

        # 计算新的宽度和高度
        new_width = int(dataset.width * (dataset.res[0] / desired_cell_size))
        new_height = int(dataset.height * (dataset.res[1] / desired_cell_size))

        # 调整分辨率
        data = dataset.read(
            out_shape=(
                dataset.count,
                new_height,
                new_width
            ),
            resampling=Resampling.nearest  # 使用最近邻插值
        )

        # 创建新的变换参数，确保每个像素的尺寸为0.25度*0.25度
        new_transform = from_origin(dataset.bounds.left, dataset.bounds.top, desired_cell_size, desired_cell_size)

        # 保存重采样后的文件
        with rasterio.open(output_file, 'w', driver='GTiff', height=new_height,
                           width=new_width, count=dataset.count,
                           dtype=data.dtype, crs=dataset.crs, transform=new_transform, nodata=nodata) as dst:
            dst.write(data)

    print(f"File saved: {output_file}")

# 新的处理逻辑
desired_cell_size = 0.25  # 欲达到的每个单元格的尺寸
years = range(2010, 2101, 10)
ssps = ['ssp1','ssp2', 'ssp3', 'ssp5']
input_base_path = "L:/popden/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"
output_base_path = "L:/popdenRES/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"

# 对每个SSP和年份的数据进行重采样
for ssp in ssps:
    for year in years:
        input_file = f"{input_base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}den.tif"
        output_file = f"{output_base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}denRES.tif"
        resample_tif(input_file, output_file, desired_cell_size)
        print(f"Resampling completed for {ssp}, year: {year}")

print("All resampling processes are completed.")

In [ ]:
#对历史数据1990年人口密度进行重采样

In [ ]:
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_origin
import os

def resample_tif(input_file, output_file, desired_cell_size):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file) as dataset:
        # 获取原始数据的 nodata 值
        nodata = dataset.nodata

        # 计算新的宽度和高度
        new_width = int(dataset.width * (dataset.res[0] / desired_cell_size))
        new_height = int(dataset.height * (dataset.res[1] / desired_cell_size))

        # 调整分辨率
        data = dataset.read(
            out_shape=(
                dataset.count,
                new_height,
                new_width
            ),
            resampling=Resampling.nearest  # 使用最近邻插值
        )

        # 创建新的变换参数，确保每个像素的尺寸为0.25度*0.25度
        new_transform = from_origin(dataset.bounds.left, dataset.bounds.top, desired_cell_size, desired_cell_size)

        # 保存重采样后的文件
        with rasterio.open(output_file, 'w', driver='GTiff', height=new_height,
                           width=new_width, count=dataset.count,
                           dtype=data.dtype, crs=dataset.crs, transform=new_transform, nodata=nodata) as dst:
            dst.write(data)

    print(f"File saved: {output_file}")

# 指定文件进行重采样
input_file = "L:/popden/popdynamics-global-pop-count-time-series-estimates-1990-geotiff/popdynamics-global-pop-count-time-series-estimates_1990den.tif"
output_file = "L:/popdenRES/popdynamics-global-pop-count-time-series-estimates-1990-geotiff/popdynamics-global-pop-count-time-series-estimates_1990denRES.tif"
desired_cell_size = 0.25  # 欲达到的每个单元格的尺寸

# 执行重采样
resample_tif(input_file, output_file, desired_cell_size)

In [ ]:
#将人口密度数据转变为人口数量数据

In [ ]:
import rasterio
import numpy as np
import os

def calculate_population_count(input_file, output_file):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file) as dataset:
        # 获取原始数据的 nodata 值
        nodata = dataset.nodata

        # 读取原始人口密度数据
        population_density = dataset.read(1)

        # 创建一个与人口密度数据相同形状的数组，用于存储每个像素的面积
        pixel_area = np.zeros(population_density.shape)

        # 计算每个像素的面积（单位：平方公里）
        for i in range(pixel_area.shape[0]):
            lat = dataset.xy(i, 0)[1]  # 获取纬度
            pixel_height_km = 111.32 * 0.25  # 每个像素的高度（公里）
            pixel_width_km = 111.32 * np.cos(np.deg2rad(lat)) * 0.25  # 每个像素的宽度（公里）
            pixel_area[i, :] = pixel_height_km * pixel_width_km

        # 计算人口数量：人口密度 * 像素面积
        population_count = np.where(population_density != nodata, population_density * pixel_area, nodata)

        # 将人口数量数据类型转换为 float32
        population_count = population_count.astype('float32')

        # 保存人口数量数据
        with rasterio.open(output_file, 'w', driver='GTiff', height=dataset.height,
                           width=dataset.width, count=1,
                           dtype='float32', crs=dataset.crs, transform=dataset.transform, nodata=nodata) as dst:
            dst.write(population_count, 1)

    print(f"Population count file saved: {output_file}")

# 新的处理逻辑
years = range(2010, 2101, 10)
ssps = ['ssp1', 'ssp2', 'ssp3', 'ssp5']
input_base_path = "L:/popdenRES/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"
output_base_path = "L:/popdenREScou/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"

# 对每个SSP和年份的数据计算人口数量
for ssp in ssps:
    for year in years:
        input_file = f"{input_base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}denRES.tif"
        output_file = f"{output_base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}denREScou.tif"
        calculate_population_count(input_file, output_file)
        print(f"Population count calculated for {ssp}, year: {year}")

print("All population count calculations are completed.")

In [ ]:
#将历史数据1990年人口密度数据转变为人口数量数据，可用  偏差不对  原始数据是5289191936.0，重采样为5302173696.0，后者是前者1.002454400062561

In [ ]:
import rasterio
import numpy as np
import os

def calculate_population_count(input_file, output_file):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file) as dataset:
        # 获取原始数据的 nodata 值
        nodata = dataset.nodata

        # 读取原始人口密度数据
        population_density = dataset.read(1)

        # 创建一个与人口密度数据相同形状的数组，用于存储每个像素的面积
        pixel_area = np.zeros(population_density.shape)

        # 计算每个像素的面积（单位：平方公里）
        for i in range(pixel_area.shape[0]):
            lat = dataset.xy(i, 0)[1]  # 获取纬度
            pixel_height_km = 111.32 * 0.25  # 每个像素的高度（公里）
            pixel_width_km = 111.32 * np.cos(np.deg2rad(lat)) * 0.25  # 每个像素的宽度（公里）
            pixel_area[i, :] = pixel_height_km * pixel_width_km

        # 计算人口数量：人口密度 * 像素面积
        population_count = np.where(population_density != nodata, population_density * pixel_area, nodata)

        # 将人口数量数据类型转换为 float32
        population_count = population_count.astype('float32')

        # 保存人口数量数据
        with rasterio.open(output_file, 'w', driver='GTiff', height=dataset.height,
                           width=dataset.width, count=1,
                           dtype='float32', crs=dataset.crs, transform=dataset.transform, nodata=nodata) as dst:
            dst.write(population_count, 1)

    print(f"Population count file saved: {output_file}")

# 执行计算人口数量的函数
input_file = "L:/popdenRES/popdynamics-global-pop-count-time-series-estimates-1990-geotiff/popdynamics-global-pop-count-time-series-estimates_1990denRES.tif"
output_file = "L:/popdenREScou/popdynamics-global-pop-count-time-series-estimates-1990-geotiff/popdynamics-global-pop-count-time-series-estimates_1990denREScou.tif"

calculate_population_count(input_file, output_file)

In [ ]:
#重要！计算原始数据人口数量和重采样后的人口数量及两者比例

In [ ]:
import rasterio
import numpy as np

def calculate_population_sum(file_path):
    try:
        with rasterio.open(file_path) as src:
            data = src.read(1)  # 读取第一个波段

            # 处理 nodata 值和nan值，将它们替换为0
            nodata = src.nodata
            if nodata is not None:
                data[data == nodata] = 0
            data = np.nan_to_num(data)

            # 计算总和
            total_population = np.sum(data)
        return total_population
    except Exception as e:
        return str(e)

# 请替换为实际文件路径
file_path1 = 'L:/pop/gpw-v4-population-count-rev11_2020_15_min_tif/gpw_v4_population_count_rev11_2020_15_min.tif'
file_path2 = 'L:/popdenREScou/gpw-v4-population-count-rev11_2020_15_min_tif/gpw_v4_population_count_rev11_2020_15_min.tif'

# 计算人口总和
population_sum1 = calculate_population_sum(file_path1)
population_sum2 = calculate_population_sum(file_path2)

# 计算比值
ratio = population_sum2 / population_sum1 if population_sum1 != 0 else None

print(f"Population sum for {file_path1}: {population_sum1}")
print(f"Population sum for {file_path2}: {population_sum2}")
print(f"Ratio of population sums: {ratio}")

In [ ]:
#计算出中间年份数据，比如基于2010年和2020年数据计算出2015年数据。

In [ ]:
import rasterio
import numpy as np
import os

def calculate_intermediate_population(input_file_early, input_file_later, output_file_intermediate):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file_intermediate)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file_early) as early_data, rasterio.open(input_file_later) as later_data:
        # 读取数据
        pop_early = early_data.read(1)
        pop_later = later_data.read(1)

        # 获取nodata值
        nodata_early = early_data.nodata
        nodata_later = later_data.nodata

        # 初始化中间年份的人口数量数组，并设置初始值为nodata
        pop_intermediate = np.full(pop_early.shape, nodata_early, dtype=np.float32)

        # 只有当两个年份都不是Nodata时，计算平均值
        mask_average = (pop_early != nodata_early) & (pop_later != nodata_later)
        pop_intermediate[mask_average] = (pop_early[mask_average] + pop_later[mask_average]) / 2

        # 如果一个年份是Nodata而另一个不是，则使用非Nodata年份的0.5倍
        mask_half_early = (pop_early != nodata_early) & (pop_later == nodata_later)
        pop_intermediate[mask_half_early] = pop_early[mask_half_early] * 0.5
        mask_half_later = (pop_early == nodata_early) & (pop_later != nodata_later)
        pop_intermediate[mask_half_later] = pop_later[mask_half_later] * 0.5

        # 使用与早期数据相同的元数据保存结果
        with rasterio.open(output_file_intermediate, 'w', **early_data.meta) as dst:
            dst.write(pop_intermediate, 1)

    print(f"File saved: {output_file_intermediate}")

# 新的处理逻辑
ssps = ['ssp1', 'ssp2', 'ssp3', 'ssp5']
base_path = "L:/popdenREScou/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"
years = range(2010, 2091, 10)  # 修改至2091以包含2090

# 对每个SSP和每个十年间隔的数据进行处理
for ssp in ssps:
    for year in years:
        input_file_early = f"{base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}denREScou.tif"
        input_file_later = f"{base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year+10}denREScou.tif"
        output_file_intermediate = f"{base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year+5}denREScou.tif"
        calculate_intermediate_population(input_file_early, input_file_later, output_file_intermediate)

print("All intermediate population calculations are completed.")

In [ ]:
#气温数据和人口数据空间不匹配，比如气温数据空间范围extent的top为90，left为0，right为360，bottom为-60，
#但人口数据空间范围extent的top是83.74999999，left为-180，right为180，bottom为-55.750000001
#所以需要先基于气温数据对人口数据进行重采样
#new_width = 1440  # 按照气温数据的尺寸调整
#new_height = 600

In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np
import os

# 定义基础文件路径
base_tif_path = 'L:\\popdenREScou\\'
base_output_path = 'L:\\popdenREScou\\RE_{}\\Total\\GeoTIFF\\'  # {} 用于插入不同的情景

# 定义情景和年份
scenarios = ['ssp1', 'ssp2', 'ssp3', 'ssp5']
years = range(2010, 2101, 5)  # 从2010到2100年，每隔五年

# 重采样参数
dst_crs = 'EPSG:4326'
new_width = 1440  # 按照气温数据的尺寸调整
new_height = 600

for scenario in scenarios:
    for year in years:
        tif_file = f'{base_tif_path}{scenario}\\Total\\GeoTIFF\\{scenario}_{year}denREScou.tif'
        output_file = f'{base_output_path.format(scenario)}{scenario}_{year}denREScou_RE.tif'

        # 确保输出文件的目录存在
        output_dir = os.path.dirname(output_file)
        os.makedirs(output_dir, exist_ok=True)

        with rasterio.open(tif_file) as src:
            nodata = src.nodata
            original_transform = src.transform
            transform = rasterio.transform.from_bounds(0, -60, 360, 90, new_width, new_height)

            # 更新文件profile并重采样
            profile = src.profile
            profile.update(transform=transform, crs=dst_crs, width=new_width, height=new_height)
            
            with rasterio.open(output_file, 'w', **profile) as dst:
                reproject(
                    source=rasterio.band(src, 1),
                    destination=rasterio.band(dst, 1),
                    src_transform=original_transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

        # 以读取模式打开重采样后的数据集
        with rasterio.open(output_file, 'r+') as dst:
            dst_data = dst.read(1)
            dst_data[dst_data == nodata] = np.nan
            dst.write_band(1, dst_data)

In [ ]:
#这里发现历史人口数据L:\popdenREScou\RE_gpw-v4-population-count-rev11_1990_15_min_tif\gpw_v4_population_count_rev11_1990_15_min_RE.tif和未来人口数据L:\popdenREScou\RE_ssp1\Total\GeoTIFF\ssp1_2025denREScou_RE.tif在形状和分辨率以及nodata上具有较大差异，将导致最后计算结果的错误，为此参考2025年数据情况对历史数据1990-2020年数据进行处理

In [ ]:
import rasterio
import numpy as np
import os

# 文件夹路径
input_folder_base = r'L:\popdenREScou\RE_gpw-v4-population-count-rev11_{}_15_min_tif'
input_file_base = 'gpw_v4_population_count_rev11_{}_15_min_RE.tif'
input_folder_2025 = r'L:\popdenREScou\RE_ssp1\Total\GeoTIFF'
output_folder = r'L:\popdenREScou\RE_lishi2000_2020'

# 确保输出文件夹存在，如果不存在则创建它
os.makedirs(output_folder, exist_ok=True)

# 输入文件名
input_file_2025 = 'ssp1_2025denREScou_RE.tif'

for year in range(1990, 2021, 5):
    input_folder = input_folder_base.format(year)
    input_file = input_file_base.format(year)
    output_file = os.path.join(output_folder, input_file_base.format(year))

    # 打开当前年份的人口数据
    with rasterio.open(os.path.join(input_folder, input_file)) as src1:
        profile1 = src1.profile
        data1 = src1.read(1)

    # 打开2025年的人口数据
    with rasterio.open(os.path.join(input_folder_2025, input_file_2025)) as src2:
        profile2 = src2.profile
        data2 = src2.read(1)

    # 确保两份数据具有相同的形状和分辨率
    assert profile1['width'] == profile2['width']
    assert profile1['height'] == profile2['height']
    assert profile1['transform'] == profile2['transform']

    # 对齐两份数据
    aligned_data = np.where(data1 != src1.nodata, data1, data2)

    # 将对齐后的数据保存到新的TIFF文件
    profile1.update(count=1, nodata=np.nan)
    with rasterio.open(output_file, 'w', **profile1) as dst:
        dst.write(aligned_data, 1)

    print(f"新的TIFF文件已成功保存至 {output_file}")

In [ ]:
#对历史数据进行处理2000-2020年，这里用的是上一步根据L:\popdenREScou\RE_ssp1\Total\GeoTIFF\ssp1_2025denREScou_RE.tif新生成的数据L:\popdenREScou\RE_lishi2000_2020\gpw_v4_population_count_rev11_1990_15_min_RE.tif

In [ ]:
import xarray as xr
import rasterio
import numpy as np
import os
import pandas as pd
import calendar

def is_leap_year(year):
    """判断是否是闰年"""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def process_data(weather_scenario, population_scenario, start_year, end_year):
    threshold_file = 'L:/CMIP6-1yuzhi/hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.1.nc'
    threshold_data = xr.open_dataset(threshold_file).load()  # 预先加载阈值数据到内存

    for year in range(start_year, end_year + 1, 5):
        population_year = year - year % 5 if year % 5 <= 2 else year + (5 - year % 5)
        weather_file = f'L:/CMIP6-1NEW/{weather_scenario}/hi_day_{weather_scenario}_r1i1p1f1_gn_{year}.nc'
        population_file = f'L:/popdenREScou/RE_lishi2000_2020/gpw_v4_population_count_rev11_{population_year}_15_min_RE.tif'        
        
        weather_data = xr.open_dataset(weather_file).load()
        with rasterio.open(population_file) as src:
            population_array = src.read(1)
            population_data = xr.DataArray(
                population_array,
                dims=["lat", "lon"],
                coords={
                    "lat": np.linspace(src.bounds.top, src.bounds.bottom, src.height),
                    "lon": np.linspace(src.bounds.left, src.bounds.right, src.width)
                }
            )
            population_data = population_data.interp(lat=weather_data.lat, lon=weather_data.lon, method='nearest')

        # 为了避免直接更新带有时间标签的DataArray，我们创建一个新的DataArray来存储调整后的数据
        adjusted_weather_data = weather_data.copy(deep=True)
        adjusted_weather_data['__xarray_dataarray_variable__'][:] = 0  # 初始化为0

        # 处理闰年数据
        if is_leap_year(year):
            date_range = pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D') + pd.DateOffset(hours=12)
        else:
            date_range = pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D') + pd.DateOffset(hours=12)

        # 调整阈值数据的时间维度以匹配目标年份
        threshold_data_adj = threshold_data.reindex(time=date_range, method='nearest')

        # 计算阈值调整后的数据
        adjusted_weather_data['__xarray_dataarray_variable__'] = xr.where(
            weather_data['__xarray_dataarray_variable__'] - threshold_data_adj['__xarray_dataarray_variable__'] > 0,
            weather_data['__xarray_dataarray_variable__'] - threshold_data_adj['__xarray_dataarray_variable__'],
            0
        )

        # 计算最终结果
        result_data = adjusted_weather_data['__xarray_dataarray_variable__'] * population_data

        # 保存结果
        save_path = f'L:/jisuan18.3_0.1/{weather_scenario}/'
        os.makedirs(save_path, exist_ok=True)
        output_file_path = os.path.join(save_path, f'{weather_scenario}_{year}.nc')
        result_data.to_netcdf(output_file_path)
        print(f"文件已成功保存至 {output_file_path}")

scenarios = {'ssp126': 'ssp1', 'ssp245': 'ssp2', 'ssp370': 'ssp3', 'ssp585': 'ssp5'}
start_year = 2000
end_year = 2020

for weather_scenario, population_scenario in scenarios.items():
    process_data(weather_scenario, population_scenario, start_year, end_year)

In [ ]:
#检查气温数据和人口数据空间是否匹配
#结果是
#气温数据维度: OrderedDict([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 360), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 600)])
#气温数据经度范围: 0.125 359.875
#气温数据纬度范围: -59.875 89.875
#人口数据边界: BoundingBox(left=0.0, bottom=-60.0, right=360.0, top=90.0)
#人口数据变换矩阵: | 0.25, 0.00, 0.00|
#| 0.00,-0.27, 90.00|
#| 0.00, 0.00, 1.00|
#人口数据CRS: WGS 84
#数据集在空间上是兼容的，可以相乘。

In [ ]:
import rasterio
from netCDF4 import Dataset

# 定义文件路径
nc_file = 'L:/CMIP6-1/ssp126/tas_day_ssp126_r1i1p1f1_gn_2025.nc'
tif_file = 'L:\\popdenREScou\\RE_ssp1\\Total\\GeoTIFF\\ssp1_2025denREScou_RE.tif'

# 检查气温数据
with Dataset(nc_file, 'r') as nc:
    nc_dims = nc.dimensions
    nc_vars = nc.variables
    print("气温数据维度:", nc_dims)
    if 'lon' in nc_vars and 'lat' in nc_vars:
        lon = nc_vars['lon'][:]
        lat = nc_vars['lat'][:]
        print("气温数据经度范围:", lon.min(), lon.max())
        print("气温数据纬度范围:", lat.min(), lat.max())

# 检查人口数据
with rasterio.open(tif_file) as src:
    tif_bounds = src.bounds
    tif_transform = src.transform
    print("人口数据边界:", tif_bounds)
    print("人口数据变换矩阵:", tif_transform)

    # 检查CRS是否匹配
    if src.crs.to_string() == 'EPSG:4326':
        print("人口数据CRS: WGS 84")
    else:
        print("人口数据CRS:", src.crs)

# 对比两个数据集以判断是否可以相乘
# 这仅仅是一个简单的对比，可能需要更详细的分析
if lon.min() >= tif_bounds.left and lon.max() <= tif_bounds.right and \
   lat.min() >= tif_bounds.bottom and lat.max() <= tif_bounds.top:
    print("数据集在空间上是兼容的，可以相乘。")
else:
    print("数据集在空间上不兼容，无法直接相乘。")

In [ ]:
#计算气温数据和人口数据乘积-预测数据

In [ ]:
import xarray as xr
import rasterio
import numpy as np
import time
import os

def process_data(weather_scenario, population_scenario, start_year, end_year):
    for year in range(start_year, end_year + 1):
        # 根据年份确定人口数据年份
        population_year = year - year % 5 if year % 5 <= 2 else year + (5 - year % 5)

        # 构建气象和人口数据的文件路径
        weather_file = f'L:/CMIP6-1new/{weather_scenario}/tas_day_{weather_scenario}_r1i1p1f1_gn_{year}.nc'
        population_file = f'L:/popdenREScou/RE_{population_scenario}/Total/GeoTIFF/{population_scenario}_{population_year}denREScou_RE.tif'

        # 读取气象数据
        weather_data = xr.open_dataset(weather_file)

        # 使用rasterio读取人口数据
        with rasterio.open(population_file) as src:
            population_array = src.read(1)
            population_data = xr.DataArray(
                population_array,
                dims=["y", "x"],
                coords={
                    "y": np.linspace(src.bounds.top, src.bounds.bottom, src.height),
                    "x": np.linspace(src.bounds.left, src.bounds.right, src.width)
                }
            )
            population_data = population_data.interp(y=weather_data['lat'], x=weather_data['lon'], method='nearest')

        # 调整气象数据
        adjusted_weather_data = weather_data['tas'] - 293.15
        adjusted_weather_data = adjusted_weather_data.clip(min=0)

        # 计算乘积
        result_data = adjusted_weather_data * population_data

        # 确保目标文件夹存在
        save_path = f'L:/jisuan20/{weather_scenario}/'
        os.makedirs(save_path, exist_ok=True)

        # 保存结果
        result_data.to_netcdf(f'{save_path}{weather_scenario}_{year}.nc')

# 循环处理每个情景和年份
scenarios = {'ssp126': 'ssp1', 'ssp245': 'ssp2', 'ssp370': 'ssp3', 'ssp585': 'ssp5'}
start_year = 2015
end_year = 2100

# 记录开始时间
start_time = time.time()

for weather_scenario, population_scenario in scenarios.items():
    process_data(weather_scenario, population_scenario, start_year, end_year)

# 计算并打印总运行时间
end_time = time.time()
total_time = end_time - start_time
hours, rem = divmod(total_time, 3600)
minutes, seconds = divmod(rem, 60)
print(f"Total time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds")

In [ ]:
#修改！计算气温数据和人口数据乘积时 不用固定阈值，而用新阈值 hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.1.nc

In [ ]:
import xarray as xr
import rasterio
import numpy as np
import os
import pandas as pd
import calendar

def is_leap_year(year):
    """判断是否是闰年"""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def process_data(weather_scenario, population_scenario, start_year, end_year):
    threshold_file = 'L:/CMIP6-1yuzhi/hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.1.nc'
    threshold_data = xr.open_dataset(threshold_file).load()  # 预先加载阈值数据到内存

    for year in range(start_year, end_year + 1, 5):
        population_year = year - year % 5 if year % 5 <= 2 else year + (5 - year % 5)
        weather_file = f'L:/CMIP6-1NEW/{weather_scenario}/hi_day_{weather_scenario}_r1i1p1f1_gn_{year}.nc'
        population_file = f'L:/popdenREScou/RE_{population_scenario}/Total/GeoTIFF/{population_scenario}_{population_year}denREScou_RE.tif'

        weather_data = xr.open_dataset(weather_file).load()
        with rasterio.open(population_file) as src:
            population_array = src.read(1)
            population_data = xr.DataArray(
                population_array,
                dims=["lat", "lon"],
                coords={
                    "lat": np.linspace(src.bounds.top, src.bounds.bottom, src.height),
                    "lon": np.linspace(src.bounds.left, src.bounds.right, src.width)
                }
            )
            population_data = population_data.interp(lat=weather_data.lat, lon=weather_data.lon, method='nearest')

        # 为了避免直接更新带有时间标签的DataArray，我们创建一个新的DataArray来存储调整后的数据
        adjusted_weather_data = weather_data.copy(deep=True)
        adjusted_weather_data['__xarray_dataarray_variable__'][:] = 0  # 初始化为0

        # 处理闰年数据
        if is_leap_year(year):
            date_range = pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D') + pd.DateOffset(hours=12)
        else:
            date_range = pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D') + pd.DateOffset(hours=12)

        # 调整阈值数据的时间维度以匹配目标年份
        threshold_data_adj = threshold_data.reindex(time=date_range, method='nearest')

        # 计算阈值调整后的数据
        adjusted_weather_data['__xarray_dataarray_variable__'] = xr.where(
            weather_data['__xarray_dataarray_variable__'] - threshold_data_adj['__xarray_dataarray_variable__'] > 0,
            weather_data['__xarray_dataarray_variable__'] - threshold_data_adj['__xarray_dataarray_variable__'],
            0
        )

        # 计算最终结果
        result_data = adjusted_weather_data['__xarray_dataarray_variable__'] * population_data

        # 保存结果
        save_path = f'L:/jisuan18.3_0.1/{weather_scenario}/'
        os.makedirs(save_path, exist_ok=True)
        output_file_path = os.path.join(save_path, f'{weather_scenario}_{year}.nc')
        result_data.to_netcdf(output_file_path)
        print(f"文件已成功保存至 {output_file_path}")

scenarios = {'ssp126': 'ssp1', 'ssp245': 'ssp2', 'ssp370': 'ssp3', 'ssp585': 'ssp5'}
start_year = 2025
end_year = 2100

for weather_scenario, population_scenario in scenarios.items():
    process_data(weather_scenario, population_scenario, start_year, end_year)

In [ ]:
#计算气温数据和人口数据乘积-历史数据
#历史数据的相乘比预测数据的相乘更为复杂

In [ ]:
import xarray as xr
import rasterio
import numpy as np
import time
import os

def process_data(weather_file, population_file, output_file):
    # 读取气象数据
    weather_data = xr.open_dataset(weather_file)

    # 使用rasterio读取人口数据
    with rasterio.open(population_file) as src:
        population_array = src.read(1)
        
        # 处理nodata值，如果存在
        nodata = src.nodata
        if nodata is not None:
            population_array[population_array == nodata] = np.nan

        population_data = xr.DataArray(
            population_array,
            dims=["y", "x"],
            coords={
                "y": np.linspace(src.bounds.top, src.bounds.bottom, src.height),
                "x": np.linspace(src.bounds.left, src.bounds.right, src.width)
            }
        )

        # 与气象数据中的坐标对齐
        population_data = population_data.interp(y=weather_data['lat'], x=weather_data['lon'], method='nearest')

    # 调整气象数据
    adjusted_weather_data = weather_data['tas'] - 293.15
    adjusted_weather_data = adjusted_weather_data.clip(min=0)

    # 将无穷值转换为0，但保留NaN
    adjusted_weather_data = xr.where(np.isinf(adjusted_weather_data), 0, adjusted_weather_data)
    population_data = xr.where(np.isinf(population_data), 0, population_data)

    # 计算乘积
    result_data = adjusted_weather_data * population_data

    # 确保目标文件夹存在
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # 保存结果
    result_data.to_netcdf(output_file)

# 记录开始时间
start_time = time.time()

# 处理2000-2014年的气温数据和对应年份的人口数据
for year in range(1990, 2015):
    # 根据气温数据年份确定人口数据年份
    population_year = year - year % 5 if year % 5 <= 2 else year + (5 - year % 5)

    # 设置文件路径
    weather_file = f'L:/CMIP6-1new/historical/tas_day_historical_r1i1p1f1_gn_{year}.nc'
    population_file = f'L:/popdenREScou/RE_gpw-v4-population-count-rev11_{population_year}_15_min_tif/gpw_v4_population_count_rev11_{population_year}_15_min_RE.tif'
    output_file = f'L:/jisuan20/historical/{year}.nc'

    # 处理数据
    process_data(weather_file, population_file, output_file)

# 计算并打印总运行时间
end_time = time.time()
total_time = end_time - start_time
hours, rem = divmod(total_time, 3600)
minutes, seconds = divmod(rem, 60)
print(f"Total time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds")

In [ ]:
#这里发现历史人口数据L:\popdenREScou\RE_gpw-v4-population-count-rev11_1990_15_min_tif\gpw_v4_population_count_rev11_1990_15_min_RE.tif和未来人口数据L:\popdenREScou\RE_ssp1\Total\GeoTIFF\ssp1_2025denREScou_RE.tif在形状和分辨率以及nodata上具有较大差异，将导致最后计算结果的错误，为此参考2025年数据情况对历史数据1990-2020年数据进行处理

In [ ]:
import rasterio
import numpy as np
import os

# 文件夹路径
input_folder_base = r'L:\popdenREScou\RE_gpw-v4-population-count-rev11_{}_15_min_tif'
input_file_base = 'gpw_v4_population_count_rev11_{}_15_min_RE.tif'
input_folder_2025 = r'L:\popdenREScou\RE_ssp1\Total\GeoTIFF'
output_folder = r'L:\popdenREScou\RE_lishi2000_2020'

# 确保输出文件夹存在，如果不存在则创建它
os.makedirs(output_folder, exist_ok=True)

# 输入文件名
input_file_2025 = 'ssp1_2025denREScou_RE.tif'

for year in range(1990, 2021, 5):
    input_folder = input_folder_base.format(year)
    input_file = input_file_base.format(year)
    output_file = os.path.join(output_folder, input_file_base.format(year))

    # 打开当前年份的人口数据
    with rasterio.open(os.path.join(input_folder, input_file)) as src1:
        profile1 = src1.profile
        data1 = src1.read(1)

    # 打开2025年的人口数据
    with rasterio.open(os.path.join(input_folder_2025, input_file_2025)) as src2:
        profile2 = src2.profile
        data2 = src2.read(1)

    # 确保两份数据具有相同的形状和分辨率
    assert profile1['width'] == profile2['width']
    assert profile1['height'] == profile2['height']
    assert profile1['transform'] == profile2['transform']

    # 对齐两份数据
    aligned_data = np.where(data1 != src1.nodata, data1, data2)

    # 将对齐后的数据保存到新的TIFF文件
    profile1.update(count=1, nodata=np.nan)
    with rasterio.open(output_file, 'w', **profile1) as dst:
        dst.write(aligned_data, 1)

    print(f"新的TIFF文件已成功保存至 {output_file}")

In [ ]:
#对历史数据进行处理2000-2020年，这里用的是上一步根据L:\popdenREScou\RE_ssp1\Total\GeoTIFF\ssp1_2025denREScou_RE.tif新生成的数据L:\popdenREScou\RE_lishi2000_2020\gpw_v4_population_count_rev11_1990_15_min_RE.tif
#修改！计算气温数据和人口数据乘积时 不用固定阈值，而用新阈值 hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.1.nc

In [ ]:
import xarray as xr
import rasterio
import numpy as np
import os
import pandas as pd
import calendar

def is_leap_year(year):
    """判断是否是闰年"""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def process_data(weather_scenario, population_scenario, start_year, end_year):
    threshold_file = 'L:/CMIP6-1yuzhi/hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.1.nc'
    threshold_data = xr.open_dataset(threshold_file).load()  # 预先加载阈值数据到内存

    for year in range(start_year, end_year + 1, 5):
        population_year = year - year % 5 if year % 5 <= 2 else year + (5 - year % 5)
        weather_file = f'L:/CMIP6-1NEW/{weather_scenario}/hi_day_{weather_scenario}_r1i1p1f1_gn_{year}.nc'
        population_file = f'L:/popdenREScou/RE_lishi2000_2020/gpw_v4_population_count_rev11_{population_year}_15_min_RE.tif'        
        
        weather_data = xr.open_dataset(weather_file).load()
        with rasterio.open(population_file) as src:
            population_array = src.read(1)
            population_data = xr.DataArray(
                population_array,
                dims=["lat", "lon"],
                coords={
                    "lat": np.linspace(src.bounds.top, src.bounds.bottom, src.height),
                    "lon": np.linspace(src.bounds.left, src.bounds.right, src.width)
                }
            )
            population_data = population_data.interp(lat=weather_data.lat, lon=weather_data.lon, method='nearest')

        # 为了避免直接更新带有时间标签的DataArray，我们创建一个新的DataArray来存储调整后的数据
        adjusted_weather_data = weather_data.copy(deep=True)
        adjusted_weather_data['__xarray_dataarray_variable__'][:] = 0  # 初始化为0

        # 处理闰年数据
        if is_leap_year(year):
            date_range = pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D') + pd.DateOffset(hours=12)
        else:
            date_range = pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D') + pd.DateOffset(hours=12)

        # 调整阈值数据的时间维度以匹配目标年份
        threshold_data_adj = threshold_data.reindex(time=date_range, method='nearest')

        # 计算阈值调整后的数据
        adjusted_weather_data['__xarray_dataarray_variable__'] = xr.where(
            weather_data['__xarray_dataarray_variable__'] - threshold_data_adj['__xarray_dataarray_variable__'] > 0,
            weather_data['__xarray_dataarray_variable__'] - threshold_data_adj['__xarray_dataarray_variable__'],
            0
        )

        # 计算最终结果
        result_data = adjusted_weather_data['__xarray_dataarray_variable__'] * population_data

        # 保存结果
        save_path = f'L:/jisuan18.3_0.1/{weather_scenario}/'
        os.makedirs(save_path, exist_ok=True)
        output_file_path = os.path.join(save_path, f'{weather_scenario}_{year}.nc')
        result_data.to_netcdf(output_file_path)
        print(f"文件已成功保存至 {output_file_path}")

scenarios = {'ssp126': 'ssp1', 'ssp245': 'ssp2', 'ssp370': 'ssp3', 'ssp585': 'ssp5'}
start_year = 2000
end_year = 2020

for weather_scenario, population_scenario in scenarios.items():
    process_data(weather_scenario, population_scenario, start_year, end_year)

In [ ]:
#最后一步，基于国家shp面数据计算栅格数值和
#这一步是不是不对啊

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import time

def process_nc_data(nc_file, shp_file, output_excel):
    print("正在读取气象数据...")
    data = xr.open_dataset(nc_file)
    var_name = list(data.data_vars)[0]
    variable_data = data[var_name]
    
    print("正在读取矢量地图并重投影...")
    countries = gpd.read_file(shp_file)
    countries = countries.to_crs(epsg=4326)
    
    print("正在处理数据...")
    final_results = pd.DataFrame()

    for time_step in variable_data.time:
        subset = variable_data.sel(time=time_step).to_dataframe().reset_index()
        points_gdf = gpd.GeoDataFrame(subset, geometry=gpd.points_from_xy(subset.lon, subset.lat))
        points_gdf.crs = 'epsg:4326'
        joined = gpd.sjoin(points_gdf, countries, how="inner", op="within")
        
        # 计算每个国家的栅格数据总和
        country_sums = joined.groupby('FCNAME')[var_name].sum()
        country_sums.name = pd.to_datetime(time_step.values).strftime('%Y-%m-%d')
        final_results = final_results.append(country_sums)

    print("数据处理完成。")

    print("正在格式化结果并保存为Excel...")
    final_results = final_results.T
    final_results.to_excel(output_excel)

    print("计算完成，结果已保存至", output_excel)

# 文件路径
nc_file = "L:/jisuan18.3/ssp126/ssp126_2030.nc"
shp_file = "L:/gis底图/countryBN/countryBN.shp"
output_excel = "L:/result18.3/ssp126/ssp126_2030.xlsx"

# 记录时间并运行函数
start_time = time.time()
process_nc_data(nc_file, shp_file, output_excel)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"总运行时间：{elapsed_time:.2f} 秒")

In [ ]:
#对2010-2100年进行重采样处理，原来数据的空间精度为0.125度*0.125度，重采样后的数据精度为0.25度*0.25度
#这是直接重采样的代码，现在不需要了

In [ ]:
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_origin
import os

def resample_tif(input_file, output_file, desired_cell_size):
    # 检查输出文件路径是否存在，如果不存在则创建
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_file) as dataset:
        # 获取原始数据的 nodata 值
        nodata = dataset.nodata

        # 计算新的宽度和高度
        new_width = int(dataset.width * (dataset.res[0] / desired_cell_size))
        new_height = int(dataset.height * (dataset.res[1] / desired_cell_size))

        # 调整分辨率
        data = dataset.read(
            out_shape=(
                dataset.count,
                new_height,
                new_width
            ),
            resampling=Resampling.nearest  # 使用最近邻插值
        )

        # 创建新的变换参数，确保每个像素的尺寸为0.25度*0.25度
        new_transform = from_origin(dataset.bounds.left, dataset.bounds.top, desired_cell_size, desired_cell_size)

        # 保存重采样后的文件
        with rasterio.open(output_file, 'w', driver='GTiff', height=new_height,
                           width=new_width, count=dataset.count,
                           dtype=data.dtype, crs=dataset.crs, transform=new_transform, nodata=nodata) as dst:
            dst.write(data)

    print(f"File saved: {output_file}")

# 新的处理逻辑
desired_cell_size = 0.25  # 欲达到的每个单元格的尺寸
years = range(2010, 2101, 10)
ssps = ['ssp1','ssp2', 'ssp3', 'ssp5']
base_path = "L:/pop/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"
output_base_path = "L:/popRES/popdynamics-1-8th-pop-base-year-projection-ssp-2000-2100-rev01-proj-"

# 对每个SSP和年份的数据进行重采样
for ssp in ssps:
    for year in years:
        input_file = f"{base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}.tif"
        output_file = f"{output_base_path}{ssp}-geotiff/{ssp}/Total/GeoTIFF/{ssp}_{year}res.tif"
        resample_tif(input_file, output_file, desired_cell_size)
        print(f"Resampling completed for {ssp}, year: {year}")

print("All resampling processes are completed.")

In [ ]:
#基于世界各国shp文件计算国家尺度气温均值 #进一步做了优化，速度加快，且可以计算运行时间。#3.73小时

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import time

# 开始计时
start_time = time.time()

# 文件路径
nc_file = r'L:\GDDP-CMIP6\ACCESS-CM2\historical\tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_1992.nc'
shp_file = r'F:\做图\世界底图\countryBN\countryBN.shp'
output_excel = r'L:\GDDP-CMIP6\ACCESS-CM2\historical_1\country_temperatures1992.xlsx'

print("正在读取气象数据...")
data = xr.open_dataset(nc_file)
tasmax = data.tasmax
print("气象数据读取完成。")

print("正在读取矢量地图并重投影...")
countries = gpd.read_file(shp_file)
countries = countries.to_crs(epsg=4326)
country_name_column = 'FCNAME'
countries.sort_values(country_name_column, inplace=True)
print("矢量地图读取并重投影完成。")

print("正在处理数据...")
# 初始化一个空的 DataFrame 用于存储最终结果
final_results = pd.DataFrame()

# 对每个时间步长处理数据
for time_step in tasmax.time:
    tasmax_subset = tasmax.sel(time=time_step)
    df_subset = tasmax_subset.to_dataframe(name='tasmax').reset_index()
    points_gdf = gpd.GeoDataFrame(df_subset, geometry=gpd.points_from_xy(df_subset.lon, df_subset.lat))
    points_gdf.crs = 'epsg:4326'
    joined = gpd.sjoin(points_gdf, countries, how="inner", op="within")
    avg_temps = joined.groupby(country_name_column)['tasmax'].mean()
    avg_temps.name = pd.to_datetime(time_step.values).strftime('%Y%m%d')
    final_results = final_results.append(avg_temps)

print("数据处理完成。")

print("正在格式化结果并保存为Excel...")
# 转置 DataFrame，以便时间步长成为列
final_results = final_results.T
final_results.to_excel(output_excel)

print("计算完成，结果已保存至", output_excel)

# 结束计时并计算总运行时间
end_time = time.time()
total_time = end_time - start_time
print(f"总运行时间：{total_time:.2f} 秒")

In [ ]:
#检查有问题的数据(历史)